# MVP - Engenharia de Dados
## Autor: Marcel Antonio Lopes Pereira

##1. Objetivo:

Este MVP tem como objetivo construir um pipeline de dados utilizando tecnologias em nuvem para buscar, coletar, modelar, carregar e analisar dados históricos do futebol de seleções masculinas. O propósito central é responder a uma questão amplamente debatida entre torcedores e especialistas:

**Existe alguma seleção nacional masculina dominante na história do futebol a ponto de ser considerada a melhor de todos os tempos?**

Para explorar essa questão, o pipeline permitirá a análise de algumas perguntas-chave, que ajudarão a identificar as seleções que se destacaram ao longo da história:

1. **Qual seleção tem o maior número de vitórias em jogos oficiais e amistosos?**  
   Identificar as seleções com o maior número de vitórias em diferentes tipos de jogos, fornecendo uma base para medir a consistência no desempenho.

2. **Qual seleção tem a melhor taxa de aproveitamento (pontos ganhos por jogo)?**  
   Avaliar a eficiência das seleções, levando em consideração o número de pontos conquistados em relação aos jogos disputados, ajudando a medir o desempenho em termos de consistência.

3. **Qual seleção tem o maior saldo de gols (gols marcados - gols sofridos) em toda a história?**  
   Analisar o saldo de gols das seleções, refletindo sua capacidade ofensiva e defensiva ao longo do tempo.

4. **Qual seleção tem o maior número de vitórias contra outras grandes seleções (top 10 do ranking da FIFA)?**  
   Analisar o desempenho das seleções contra as mais fortes equipes do mundo, refletindo sua competitividade no cenário global.

5. **Qual seleção tem a maior sequência invicta em toda a história?**  
   Identificar quais seleções mantiveram longas sequências sem derrotas, um bom indicativo de consistência e resistência ao longo do tempo.

6. **Qual seleção tem o maior número de jogos sem sofrer gols em toda a história?**  
   Analisar a capacidade defensiva das seleções, refletindo uma defesa sólida e organização tática eficaz ao longo dos anos.

7. **Quais seleções tiveram os artilheiros mais consistentes e como o desempenho desses jogadores está relacionado com o sucesso da equipe?**  
   Relacionar o desempenho individual dos artilheiros com o desempenho da seleção, ajudando a identificar como a contribuição dos goleadores impactou as vitórias e o sucesso da equipe ao longo da história.



##2. Estrutura do Pipeline

O pipeline de dados será estruturado em três camadas principais:

- **Bronze:** coleta e armazenamento dos dados brutos conforme os arquivos originais.  
- **Silver:** limpeza, padronização e transformação dos dados brutos, garantindo consistência e qualidade para as análises. Nesta etapa, serão tratados problemas como dados ausentes, duplicidades e formatação inadequada.  
- **Gold:** modelagem analítica dos dados para responder às perguntas de negócio, além da otimização para consultas complexas, garantindo eficiência na extração de insights e facilitando a análise exploratória e preditiva.

A análise final dos dados será realizada em um notebook específico, onde todas as perguntas levantadas serão respondidas. A partir dos resultados obtidos, será possível avaliar, com base em dados, quais seleções se destacam historicamente e se alguma delas pode ser considerada a mais dominante de todos os tempos.





## 3. Busca dos dados:

Na busca dos dados necessários para solução do problema, foi escolhido o Kaggle como fonte para obtenção dos mesmos. Nesse processo, foram escolhidos dois conjuntos de dados que são descritos a seguir.

O primeiro conjunto é referente aos resultados de partidas entre seleções no período entre 1872 e 2025. Nesses dados não estão incluídos os resultados de partidas em Jogos Olímípicos, com seleções nacionais B, sub-23 e seleções com melhores jogadores de ligas. O link de acesso a esse conjunto de dados é o seguinte:
https://www.kaggle.com/datasets/martj42/international-football-results-from-1872-to-2017

Esses dados foram publicados pelo usuário Mart Jürisoo e são de domínio público (CC0: Public Domain). O conjunto possui 4 arquivos do tipo csv: results.csv (resultados das partidas); shootouts.csv (disputas de penalties); goalscorers.csv (marcadores de gols); former_names.csv (nomes anteriores das equipes).

O segundo conjunto é referente ao Ranking da FIFA de seleções no período de 1992 a 2024. Estão listadas as posições dos times e suas mudanças ao longo do tempo dentro desse intervalo. O link de acesso a esse conjunto de dados é o seguinte:
https://www.kaggle.com/datasets/cashncarry/fifaworldranking

Esses dados forma publicados pelo usuário Alex e também são de domínio público (CC0: Public Domain). Apesar do conjunto possuir 3 arquivos do tipo csv, somente foi utilizado um deles nesse trabalho: fifa_ranking-2024-06-20.csv (Lista de Rankings até 20-06-24).

## 4. Coleta, modelagem e carga dos dados:

Após identificar os dados necessários para o problema, é realizada a coleta e a carga dos mesmos para o ambiente de nuvem. Foi utilizado nesse processo o sistema de armazenamento do Datrabricks com utilização do framework Spark para carga dos dados.


In [0]:
# Importando as bibliotecas necessárias para o trabalho
import pandas as pd

### 4.1 Camada Bronze

Na camada bronze, são importados os dados no seu formato bruto a partir dos arquivos csv de origem. São realizados scripts relativos a criação da database onde serão armazenados as tabelas da camada bronze.

In [0]:
%sql
-- Desfazendo o database bronze para ser "recriado"
DROP DATABASE IF EXISTS bronze CASCADE;

In [0]:
%sql 
-- Criando o database bronze
CREATE DATABASE bronze;

Após a criação do database, os arquivos CSV são carregados utilizando a biblioteca Pandas para gerar DataFrames a partir dos respectivos arquivos. Foi realizado download dos arquivos no Kaggle e os mesmos foram salvos no mesmo repositório onde esse Notebook está alocado. Em seguida, esses DataFrames são convertidos para o formato Spark e armazenados como tabelas no sistema de armazenamento em nuvem do Databricks, dentro da camada Bronze do database criado.

In [0]:
# Dicionário com os nomes das tabelas e URLs dos arquivos
csv_files = {
    "results": "https://raw.githubusercontent.com/MarcelLopes7/Engenharia-de-Dados/refs/heads/main/results.csv",
    "shootouts": "https://raw.githubusercontent.com/MarcelLopes7/Engenharia-de-Dados/refs/heads/main/shootouts.csv",
    "goalscorers": "https://raw.githubusercontent.com/MarcelLopes7/Engenharia-de-Dados/refs/heads/main/goalscorers.csv",
    "former_names": "https://raw.githubusercontent.com/MarcelLopes7/Engenharia-de-Dados/refs/heads/main/former_names.csv",
    "fifa_ranking": "https://raw.githubusercontent.com/MarcelLopes7/Engenharia-de-Dados/refs/heads/main/fifa_ranking-2024-06-20.csv"
}

# Loop para carregar, converter e salvar cada arquivo
for table_name, url in csv_files.items():
    # Carregar o CSV como DataFrame Pandas
    df = pd.read_csv(url)
    
    # Converter para DataFrame Spark
    spark_df = spark.createDataFrame(df)
    
    # Salvar como tabela na camada bronze
    spark_df.write.mode("overwrite").saveAsTable(f"bronze.{table_name}")

    print(f"Tabela bronze.{table_name} carregada com sucesso.")


Tabela bronze.results carregada com sucesso.
Tabela bronze.shootouts carregada com sucesso.
Tabela bronze.goalscorers carregada com sucesso.
Tabela bronze.former_names carregada com sucesso.
Tabela bronze.fifa_ranking carregada com sucesso.


Após a execução dos script acima, a camada bronze foi construída com as tabelas. Em seguida, são realizadas verificações Spark SQL para confirmar a criação das tabelas. 


In [0]:
# Lista de tabelas que foram criadas
tables = ["results", "shootouts", "goalscorers", "former_names", "fifa_ranking"]

# Exibindo as 10 primeiras linhas de cada tabela
for table in tables:
    print(f"🔍 Exibindo 10 primeiras linhas da tabela bronze.{table}")
    display(spark.sql(f"SELECT * FROM bronze.{table} LIMIT 10"))

# Exibindo o esquema de cada tabela
for table in tables:
    print(f"📊 Esquema da tabela bronze.{table}")
    spark.sql(f"DESCRIBE TABLE bronze.{table}").show()


🔍 Exibindo 10 primeiras linhas da tabela bronze.results


date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,false
1873-03-08,England,Scotland,4,2,Friendly,London,England,false
1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,false
1875-03-06,England,Scotland,2,2,Friendly,London,England,false
1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,false
1876-03-25,Scotland,Wales,4,0,Friendly,Glasgow,Scotland,false
1877-03-03,England,Scotland,1,3,Friendly,London,England,false
1877-03-05,Wales,Scotland,0,2,Friendly,Wrexham,Wales,false
1878-03-02,Scotland,England,7,2,Friendly,Glasgow,Scotland,false
1878-03-23,Scotland,Wales,9,0,Friendly,Glasgow,Scotland,false


🔍 Exibindo 10 primeiras linhas da tabela bronze.shootouts


date,home_team,away_team,winner,first_shooter
1967-08-22,India,Taiwan,Taiwan,null
1971-11-14,South Korea,Vietnam Republic,South Korea,null
1972-05-07,South Korea,Iraq,Iraq,null
1972-05-17,Thailand,South Korea,South Korea,null
1972-05-19,Thailand,Cambodia,Thailand,null
1973-04-21,Senegal,Ghana,Ghana,null
1973-06-14,Guinea,Mali,Guinea,null
1973-06-14,Mauritius,Tanzania,Mauritius,null
1973-07-26,Malaysia,Kuwait,Malaysia,null
1973-07-26,Cambodia,Singapore,Singapore,null


🔍 Exibindo 10 primeiras linhas da tabela bronze.goalscorers


...

🔍 Exibindo 10 primeiras linhas da tabela bronze.former_names


current,former,start_date,end_date
Benin,Dahomey,1959-11-08,1975-11-30
Burkina Faso,Upper Volta,1960-04-14,1984-08-04
Curaçao,Netherlands Antilles,1957-03-03,2010-10-10
Czechoslovakia,Bohemia,1903-04-05,1919-01-01
Czechoslovakia,Bohemia and Moravia,1939-01-01,1945-05-01
Czechoslovakia,Representation of Czechs and Slovaks,1993-03-24,1993-11-17
DR Congo,Belgian Congo,1948-05-25,1956-01-02
DR Congo,Congo-Léopoldville,1963-04-12,1964-07-19
DR Congo,Congo-Kinshasa,1965-01-09,1970-11-24
DR Congo,Zaïre,1971-01-10,1997-04-27


🔍 Exibindo 10 primeiras linhas da tabela bronze.fifa_ranking


rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date
140.0,Brunei Darussalam,BRU,2.0,0.0,140,AFC,1992-12-31
33.0,Portugal,POR,38.0,0.0,33,UEFA,1992-12-31
32.0,Zambia,ZAM,38.0,0.0,32,CAF,1992-12-31
31.0,Greece,GRE,38.0,0.0,31,UEFA,1992-12-31
30.0,Algeria,ALG,39.0,0.0,30,CAF,1992-12-31
29.0,Yugoslavia,YUG,39.0,0.0,29,UEFA,1992-12-31
28.0,Wales,WAL,40.0,0.0,28,UEFA,1992-12-31
27.0,Côte d'Ivoire,CIV,41.0,0.0,27,CAF,1992-12-31
34.0,Austria,AUT,38.0,0.0,34,UEFA,1992-12-31
26.0,Bulgaria,BUL,41.0,0.0,26,UEFA,1992-12-31


📊 Esquema da tabela bronze.results
+----------+---------+-------+
|  col_name|data_type|comment|
+----------+---------+-------+
|      date|   string|   null|
| home_team|   string|   null|
| away_team|   string|   null|
|home_score|   bigint|   null|
|away_score|   bigint|   null|
|tournament|   string|   null|
|      city|   string|   null|
|   country|   string|   null|
|   neutral|  boolean|   null|
+----------+---------+-------+

📊 Esquema da tabela bronze.shootouts
+-------------+---------+-------+
|     col_name|data_type|comment|
+-------------+---------+-------+
|         date|   string|   null|
|    home_team|   string|   null|
|    away_team|   string|   null|
|       winner|   string|   null|
|first_shooter|   string|   null|
+-------------+---------+-------+

📊 Esquema da tabela bronze.goalscorers
+---------+---------+-------+
| col_name|data_type|comment|
+---------+---------+-------+
|     date|   string|   null|
|home_team|   string|   null|
|away_team|   string|   null

Nesta etapa do projeto, foi concluída a ingestão dos dados brutos provenientes de diversos arquivos CSV históricos relacionados às seleções nacionais de futebol, utilizando o Spark no Databricks. Com a criação de tabelas na camada bronze, os dados foram armazenados em sua forma original, preservando todas as informações com a maior fidelidade possível.

Agora que a camada bronze está criada e os dados estão prontos para análise, a próxima etapa do trabalho será a construção da camada silver, onde será feita a limpeza, transformação e integração dos dados para garantir que estejam no formato adequado para análises mais complexas.

A camada silver será responsável por aprimorar a qualidade dos dados, identificando e corrigindo inconsistências, transformando variáveis e aplicando quaisquer outras transformações necessárias para garantir que os dados estejam prontos para a análise e construção de modelos na camada gold.

Com isso, segue-se para a próxima fase do pipeline de dados, garantindo que os dados estejam cada vez mais preparados para responder às questões relacionadas à dominância das seleções nacionais no futebol.

##  4.2 Camada Silver

A camada silver tem como objetivo realizar um processamento mais refinado dos dados provenientes da camada bronze, aplicando transformações e limpezas essenciais para garantir que os dados estejam padronizados e prontos para análises mais complexas na camada gold.

Utilizando as tabelas da camada bronze como base, foram realizadas operações como o tratamento de nomes de seleções através da tabela former_names, além de aplicar ajustes nos formatos de dados (como conversões de tipos de variáveis) e, quando necessário, a remoção de dados nulos. As tabelas resultantes da camada silver servirão como um ponto de partida para análises mais profundas e criação de métricas na camada gold.

A seguir, serão processados os principais dados de resultados de partidas, disputas de pênaltis, marcadores de gols e o ranking FIFA para garantir que todas as informações estejam corretamente estruturadas e padronizadas. Ao final do processo, haverá tabelas com informações prontas para serem utilizadas em análises avançadas, como a performance das seleções ao longo do tempo.
Primeiramente, são executados scripts de criação do database silver.

In [0]:
%sql DROP DATABASE IF EXISTS silver CASCADE

In [0]:
%sql CREATE DATABASE silver

A partir disso, são criadas as tabelas na camada silver a partir das tabelas da camada Bronze. O processo de criação de cada uma delas é descrito a seguir.

### 4.2.1 Tabela Former Names
A primeira tabela a ser criada foi a silver.former_names. Nesta etapa do pipeline de ingestão e transformação, a tabela silver.former_names foi construída a partir da camada bronze, com o objetivo de refinar e padronizar as informações relacionadas às alterações históricas de nome das seleções nacionais. Durante o processo, foram aplicadas rotinas de limpeza que incluem a remoção de valores nulos, o tratamento de espaços em branco, a conversão explícita de campos de data para o tipo DATE e a validação de integridade temporal (start_date <= end_date). Essas transformações garantem maior confiabilidade e consistência aos dados, preparando-os para análises mais avançadas nas camadas analíticas do projeto.

In [0]:
%sql
DROP TABLE IF EXISTS silver.former_names;

CREATE TABLE silver.former_names AS
SELECT 
  TRIM(current) AS current,
  TRIM(former) AS former,
  CAST(start_date AS DATE) AS start_date,
  CAST(end_date AS DATE) AS end_date
FROM bronze.former_names
WHERE 
  current IS NOT NULL
  AND former IS NOT NULL
  AND start_date <= end_date;

num_affected_rows,num_inserted_rows


A seguir, é verificado na tabela se todo o processo foi realizado com sucesso.

In [0]:
%sql
-- Verificação de qualidade dos dados após criação da tabela
WITH checks AS (
  SELECT
    COUNT(*) AS total_records,  -- Total de registros transformados
    
    SUM(CASE 
        WHEN silver.start_date IS NULL OR silver.end_date IS NULL 
        THEN 1 ELSE 0 END) AS null_dates,  -- Datas convertidas corretamente
    
    SUM(CASE 
        WHEN silver.current IS NULL OR silver.former IS NULL 
        THEN 1 ELSE 0 END) AS null_names,  -- Campos nulos nos nomes
    
    SUM(CASE 
        WHEN silver.start_date > silver.end_date 
        THEN 1 ELSE 0 END) AS invalid_date_ranges,  -- Consistência temporal
    
    (SELECT COUNT(*) FROM bronze.former_names) AS source_count,  -- Verificação de volume da fonte
    (SELECT COUNT(DISTINCT former) FROM bronze.former_names) AS source_unique_names  -- 👍 Insight adicional
  FROM silver.former_names silver
)

SELECT
  total_records,
  source_count,
  source_unique_names,
  null_dates,
  null_names,
  invalid_date_ranges,
  
  CASE
    WHEN null_dates = 0 AND null_names = 0 AND invalid_date_ranges = 0 
    THEN '✅ Validações básicas passaram'
    ELSE '❌ Problemas encontrados'
  END AS basic_validation,
  
  CASE
    WHEN total_records <= source_count AND total_records > 0
    THEN '✅ Contagem de registros OK'
    ELSE '❌ Problema na contagem de registros'
  END AS count_validation
FROM checks;


total_records,source_count,source_unique_names,null_dates,null_names,invalid_date_ranges,basic_validation,count_validation
34,34,34,0,0,0,✅ Validações básicas passaram,✅ Contagem de registros OK


Visualização da tabela criada.

In [0]:
%sql
SELECT *
FROM silver.former_names

current,former,start_date,end_date
Benin,Dahomey,1959-11-08,1975-11-30
Burkina Faso,Upper Volta,1960-04-14,1984-08-04
Curaçao,Netherlands Antilles,1957-03-03,2010-10-10
Czechoslovakia,Bohemia,1903-04-05,1919-01-01
Czechoslovakia,Bohemia and Moravia,1939-01-01,1945-05-01
Czechoslovakia,Representation of Czechs and Slovaks,1993-03-24,1993-11-17
DR Congo,Belgian Congo,1948-05-25,1956-01-02
DR Congo,Congo-Léopoldville,1963-04-12,1964-07-19
DR Congo,Congo-Kinshasa,1965-01-09,1970-11-24
DR Congo,Zaïre,1971-01-10,1997-04-27


Alterando o nome de um país para evitar conflito de nome com a tabela results.

In [0]:
%sql
UPDATE silver.former_names
SET current = 'Czech Republic'
WHERE current = 'Czechoslovakia';

num_affected_rows
3


In [0]:
%sql
INSERT INTO silver.former_names (current, former, start_date, end_date)
VALUES ('Czech Republic', 'Czechoslovakia', '1993-11-17', NULL);

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
SELECT * FROM silver.former_names WHERE current = 'Czech Republic'

current,former,start_date,end_date
Czech Republic,Bohemia,1903-04-05,1919-01-01
Czech Republic,Bohemia and Moravia,1939-01-01,1945-05-01
Czech Republic,Representation of Czechs and Slovaks,1993-03-24,1993-11-17
Czech Republic,Czechoslovakia,1993-11-17,null


Desse modo, essa tabela será agregada com as outras posteriormente de modo que os times estejam registrados com um nome único por todo o período de tempo.

### 4.2.2 Tabela Results
A próxima tabela a ser criada é a tabela silver.results. Nesta etapa, foi criada a tabela silver.results, responsável por armazenar os resultados das partidas internacionais de forma padronizada e enriquecida. O fluxo de construção da tabela silver.results inicia com os dados brutos da camada Bronze (bronze.results) e aplica uma série de transformações essenciais para garantir consistência, qualidade e enriquecimento dos dados. A partir da base bruta bronze.results, foram aplicadas diversas transformações: conversão da data para o tipo DATE, padronização dos nomes das seleções por meio da junção com a tabela silver.former_names, limpeza do campo tournament, tipagem correta dos placares e criação de um identificador único para cada partida (match_id). Além disso, registros com pontuação nula foram descartados, assegurando que apenas jogos concluídos estejam presentes na camada silver. 



In [0]:
%sql
DROP TABLE IF EXISTS silver.results;

CREATE TABLE silver.results AS
SELECT 
  date(r.date) AS match_date,
  COALESCE(fn_home.current, r.home_team) AS home_team,
  COALESCE(fn_away.current, r.away_team) AS away_team,
  CAST(r.home_score AS INT) AS home_score,
  CAST(r.away_score AS INT) AS away_score,
  trim(r.tournament) AS tournament,
  r.city,
  r.country,
  CAST(r.neutral AS BOOLEAN) AS is_neutral,
  CONCAT(
    CAST(date(r.date) AS STRING), 
    '_', 
    COALESCE(fn_home.current, r.home_team), 
    '_', 
    COALESCE(fn_away.current, r.away_team), 
    '_', 
    CAST(r.home_score AS STRING), 
    '_', 
    CAST(r.away_score AS STRING)
  ) AS match_id  -- Criando match_id APÓS a substituição dos nomes
FROM bronze.results r
LEFT JOIN silver.former_names fn_home 
  ON TRIM(r.home_team) = TRIM(fn_home.former)
LEFT JOIN silver.former_names fn_away 
  ON TRIM(r.away_team) = TRIM(fn_away.former)
WHERE r.home_score IS NOT NULL;


num_affected_rows,num_inserted_rows


Após as transformações para criação da tabela, é realizada a validação das mesmas.

In [0]:
%sql
WITH checks AS (
  SELECT
    COUNT(*) AS total_records,

    -- Validação de campos obrigatórios
    SUM(CASE WHEN match_date IS NULL THEN 1 ELSE 0 END) AS null_dates,
    SUM(CASE WHEN home_team IS NULL OR away_team IS NULL THEN 1 ELSE 0 END) AS null_teams,
    SUM(CASE WHEN home_score IS NULL OR away_score IS NULL THEN 1 ELSE 0 END) AS null_scores,
    SUM(CASE WHEN match_id IS NULL OR match_id = '' THEN 1 ELSE 0 END) AS null_match_id,
    
    -- Placar deve ser zero ou positivo
    SUM(CASE WHEN home_score < 0 OR away_score < 0 THEN 1 ELSE 0 END) AS negative_scores,
    
    -- Verificação de duplicidade no match_id
    COUNT(DISTINCT match_id) AS unique_match_ids,

    -- Total original para comparação (excluindo scores nulos como no filtro do SELECT)
    (SELECT COUNT(*) FROM bronze.results WHERE home_score IS NOT NULL) AS source_count
  FROM silver.results
)

SELECT
  total_records,
  unique_match_ids,
  source_count,
  null_dates,
  null_teams,
  null_scores,
  null_match_id,
  negative_scores,

  -- Relatório de integridade
  CASE
    WHEN null_dates = 0 AND null_teams = 0 AND null_scores = 0 
         AND null_match_id = 0 AND negative_scores = 0
    THEN '✅ Validações básicas passaram'
    ELSE '❌ Problemas encontrados nos dados'
  END AS basic_validation,

  -- Validação de ID único
  CASE
    WHEN total_records = unique_match_ids 
    THEN '✅ match_id único para cada partida'
    ELSE '❌ match_id duplicado detectado'
  END AS match_id_validation,

  -- Validação de volume
  CASE
    WHEN total_records = source_count 
    THEN '✅ Volume consistente com a fonte (após filtro)'
    ELSE '❌ Divergência na contagem de registros'
  END AS count_validation

FROM checks;


total_records,unique_match_ids,source_count,null_dates,null_teams,null_scores,null_match_id,negative_scores,basic_validation,match_id_validation,count_validation
48207,48190,48207,0,0,0,0,0,✅ Validações básicas passaram,❌ match_id duplicado detectado,✅ Volume consistente com a fonte (após filtro)


In [0]:
%sql
SELECT *
FROM silver.results
WHERE match_id IN (
  SELECT match_id
  FROM silver.results
  GROUP BY match_id
  HAVING COUNT(*) > 1
)
ORDER BY match_id, match_date;


match_date,home_team,away_team,home_score,away_score,tournament,city,country,is_neutral,match_id
1923-05-22,China PR,Philippines,3,0,Friendly,Osaka,Japan,true,1923-05-22_China PR_Philippines_3_0
1923-05-22,China PR,Philippines,3,0,Far Eastern Championship Games,Osaka,Japan,true,1923-05-22_China PR_Philippines_3_0
1923-05-24,Japan,China PR,1,5,Friendly,Osaka,Japan,false,1923-05-24_Japan_China PR_1_5
1923-05-24,Japan,China PR,1,5,Far Eastern Championship Games,Osaka,Japan,false,1923-05-24_Japan_China PR_1_5
1925-05-22,Philippines,China PR,1,5,Friendly,Manila,Philippines,false,1925-05-22_Philippines_China PR_1_5
1925-05-22,Philippines,China PR,1,5,Far Eastern Championship Games,Manila,Philippines,false,1925-05-22_Philippines_China PR_1_5
1927-08-27,China PR,Japan,5,1,Friendly,Shanghai,China PR,false,1927-08-27_China PR_Japan_5_1
1927-08-27,China PR,Japan,5,1,Far Eastern Championship Games,Shanghai,China PR,false,1927-08-27_China PR_Japan_5_1
1927-08-29,Japan,Philippines,2,1,Friendly,Shanghai,China PR,true,1927-08-29_Japan_Philippines_2_1
1927-08-29,Japan,Philippines,2,1,Far Eastern Championship Games,Shanghai,China PR,true,1927-08-29_Japan_Philippines_2_1


Durante a verificação dos dados na tabela silver.results, foram identificadas duplicatas de partidas com os mesmos times, placar e data, mas listadas com dois nomes de torneio distintos: "Friendly" e o nome real do campeonato (como "Far Eastern Championship Games" ou "African Friendship Games").

Essas duplicações ocorrem porque o campo match_id é gerado com base apenas na data, equipes e placar, desconsiderando o tipo de torneio. Como o nome do torneio real é mais informativo e relevante para análises futuras, optou-se por remover os registros do tipo "Friendly" quando uma partida equivalente já estiver registrada com o nome correto do torneio.

A seguir, foi utilizada uma instrução DELETE com subconsulta EXISTS para garantir a remoção segura e controlada desses casos duplicados:

In [0]:
%sql
DELETE FROM silver.results r
WHERE r.tournament = 'Friendly'
  AND EXISTS (
    SELECT 1
    FROM silver.results s
    WHERE 
      s.match_date = r.match_date
      AND s.home_team = r.home_team
      AND s.away_team = r.away_team
      AND s.home_score = r.home_score
      AND s.away_score = r.away_score
      AND s.tournament <> 'Friendly'
  );

num_affected_rows
17


Verificando a exclusão das linhas duplicadas.

In [0]:
%sql
SELECT *
FROM silver.results r1
JOIN silver.results r2
  ON r1.match_date = r2.match_date
 AND r1.home_team = r2.home_team
 AND r1.away_team = r2.away_team
 AND r1.home_score = r2.home_score
 AND r1.away_score = r2.away_score
WHERE r1.tournament = 'Friendly'
  AND r2.tournament != 'Friendly';

match_date,home_team,away_team,home_score,away_score,tournament,city,country,is_neutral,match_id,match_date,home_team,away_team,home_score,away_score,tournament,city,country,is_neutral,match_id


Para melhorar a documentação dos dados e facilitar sua interpretação, foram adicionados comentários descritivos à tabela silver.results e suas colunas. Isso garante que futuras consultas e análises possam ser feitas com clareza sobre o significado e as restrições dos dados.

In [0]:
%sql
-- Comentário geral da tabela
ALTER TABLE silver.results 
SET TBLPROPERTIES (
  'comment' = 'Tabela de resultados de partidas de futebol internacionais. Contém dados brutos padronizados para uso na camada Gold. Fonte: bronze.results.'
);

-- 🔍 Comentários das colunas (um comando por coluna)
ALTER TABLE silver.results CHANGE COLUMN match_id match_id STRING COMMENT 
  'Código único de identificação da partida. Formato: YYYY-MM-DD_HomeTeam_AwayTeam_HomeScore_AwayScore.';
  
ALTER TABLE silver.results CHANGE COLUMN match_date match_date DATE COMMENT 
  'Data da partida no formato YYYY-MM-DD. Domínio: Datas entre 1872-11-30 (primeiro jogo internacional) e a data atual.';
  
ALTER TABLE silver.results CHANGE COLUMN home_team home_team STRING COMMENT 
  'Nome do time da casa padronizado. Domínio: Valores da tabela silver.teams (ex: "Brazil", "Germany").';
  
ALTER TABLE silver.results CHANGE COLUMN away_team away_team STRING COMMENT 
  'Nome do time visitante padronizado. Domínio: Valores da tabela silver.teams.';
  
ALTER TABLE silver.results CHANGE COLUMN home_score home_score INT COMMENT 
  'Gols marcados pelo time da casa. Domínio: Inteiro ≥ 0.';
  
ALTER TABLE silver.results CHANGE COLUMN away_score away_score INT COMMENT 
  'Gols marcados pelo time visitante. Domínio: Inteiro ≥ 0.';
  
ALTER TABLE silver.results CHANGE COLUMN tournament tournament STRING COMMENT 
  'Nome do torneio sem formatação adicional. Domínio: Texto livre (ex: "FIFA World Cup", "Copa América").';
  
ALTER TABLE silver.results CHANGE COLUMN city city STRING COMMENT 
  'Cidade onde a partida foi realizada. Domínio: Texto livre (ex: "Rio de Janeiro", "Berlin").';
  
ALTER TABLE silver.results CHANGE COLUMN country country STRING COMMENT 
  'País onde a partida foi realizada. Domínio: Nomes de países ISO 3166 (ex: "Brazil", "Spain").';
  
ALTER TABLE silver.results CHANGE COLUMN is_neutral is_neutral BOOLEAN COMMENT 
  'Indica se o jogo foi em local neutro. Domínio: TRUE/FALSE.';


In [0]:
%sql
DESCRIBE TABLE silver.results;

col_name,data_type,comment
match_date,date,Data da partida no formato YYYY-MM-DD. Domínio: Datas entre 1872-11-30 (primeiro jogo internacional) e a data atual.
home_team,string,"Nome do time da casa padronizado. Domínio: Valores da tabela silver.teams (ex: ""Brazil"", ""Germany"")."
away_team,string,Nome do time visitante padronizado. Domínio: Valores da tabela silver.teams.
home_score,int,Gols marcados pelo time da casa. Domínio: Inteiro ≥ 0.
away_score,int,Gols marcados pelo time visitante. Domínio: Inteiro ≥ 0.
tournament,string,"Nome do torneio sem formatação adicional. Domínio: Texto livre (ex: ""FIFA World Cup"", ""Copa América"")."
city,string,"Cidade onde a partida foi realizada. Domínio: Texto livre (ex: ""Rio de Janeiro"", ""Berlin"")."
country,string,"País onde a partida foi realizada. Domínio: Nomes de países ISO 3166 (ex: ""Brazil"", ""Spain"")."
is_neutral,boolean,Indica se o jogo foi em local neutro. Domínio: TRUE/FALSE.
match_id,string,Código único de identificação da partida. Formato: YYYY-MM-DD_HomeTeam_AwayTeam_HomeScore_AwayScore.


É verificada a criação da tabela.

In [0]:
%sql
-- Verificar a tabela silver.results
SELECT * 
FROM silver.results 
LIMIT 20

match_date,home_team,away_team,home_score,away_score,tournament,city,country,is_neutral,match_id
1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,false,1872-11-30_Scotland_England_0_0
1873-03-08,England,Scotland,4,2,Friendly,London,England,false,1873-03-08_England_Scotland_4_2
1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,false,1874-03-07_Scotland_England_2_1
1875-03-06,England,Scotland,2,2,Friendly,London,England,false,1875-03-06_England_Scotland_2_2
1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,false,1876-03-04_Scotland_England_3_0
1876-03-25,Scotland,Wales,4,0,Friendly,Glasgow,Scotland,false,1876-03-25_Scotland_Wales_4_0
1877-03-03,England,Scotland,1,3,Friendly,London,England,false,1877-03-03_England_Scotland_1_3
1877-03-05,Wales,Scotland,0,2,Friendly,Wrexham,Wales,false,1877-03-05_Wales_Scotland_0_2
1878-03-02,Scotland,England,7,2,Friendly,Glasgow,Scotland,false,1878-03-02_Scotland_England_7_2
1878-03-23,Scotland,Wales,9,0,Friendly,Glasgow,Scotland,false,1878-03-23_Scotland_Wales_9_0


### 4.2.3 Tabela `silver.shootouts`
A tabela `silver.shootouts` foi criada a partir da padronização dos dados brutos presentes em `bronze.shootouts`, representando disputas de pênaltis ocorridas após o término das partidas. Para garantir integridade e consistência, cada disputa foi vinculada à tabela `silver.results` por meio do campo `match_date`, juntamente com os nomes das equipes mandante e visitante. Essa junção possibilitou a reutilização do campo `match_id` já padronizado como identificador único da partida. Os nomes das equipes foram padronizados utilizando a tabela `silver.former_names`, assegurando uniformidade na representação histórica. Além disso, valores ausentes no campo `first_shooter` foram substituídos pela string 'Unknown' para manter a completude do dado.

In [0]:
%sql
DROP TABLE IF EXISTS silver.shootouts;

CREATE TABLE silver.shootouts AS
SELECT 
  r.match_id,  -- Usando o match_id da tabela results já existente
  date(s.date) AS match_date,
  COALESCE(fn_home.current, s.home_team) AS home_team,
  COALESCE(fn_away.current, s.away_team) AS away_team,
  COALESCE(fn_winner.current, s.winner) AS winner,
  COALESCE(s.first_shooter, 'Unknown') AS first_shooter
FROM bronze.shootouts s
JOIN silver.results r
  ON r.home_team = s.home_team
  AND r.away_team = s.away_team
  AND r.match_date = date(s.date)
LEFT JOIN silver.former_names fn_home ON s.home_team = fn_home.former
LEFT JOIN silver.former_names fn_away ON s.away_team = fn_away.former
LEFT JOIN silver.former_names fn_winner ON s.winner = fn_winner.former;


num_affected_rows,num_inserted_rows


As tranformações são validadas.

In [0]:
%sql
WITH checks AS (
  SELECT
    COUNT(*) AS total_records,

    -- Validação de campos obrigatórios
    SUM(CASE WHEN match_date IS NULL THEN 1 ELSE 0 END) AS null_dates,
    SUM(CASE WHEN home_team IS NULL OR away_team IS NULL THEN 1 ELSE 0 END) AS null_teams,
    SUM(CASE WHEN winner IS NULL THEN 1 ELSE 0 END) AS null_winner,
    SUM(CASE WHEN match_id IS NULL OR match_id = '' THEN 1 ELSE 0 END) AS null_match_id,
    
    -- Verificação de duplicidade no match_id
    COUNT(DISTINCT match_id) AS unique_match_ids,

    -- Total original para comparação (base bronze)
    (SELECT COUNT(*) FROM bronze.shootouts) AS source_count
  FROM silver.shootouts
)

SELECT
  total_records,
  unique_match_ids,
  source_count,
  null_dates,
  null_teams,
  null_winner,
  null_match_id,

  -- Relatório de integridade
  CASE
    WHEN null_dates = 0 AND null_teams = 0 AND null_winner = 0 
         AND null_match_id = 0
    THEN '✅ Validações básicas passaram'
    ELSE '❌ Problemas encontrados nos dados'
  END AS basic_validation,

  -- Validação de ID único
  CASE
    WHEN total_records = unique_match_ids 
    THEN '✅ match_id único para cada disputa de pênaltis'
    ELSE '❌ match_id duplicado detectado'
  END AS match_id_validation,

  -- Validação de volume
  CASE
    WHEN total_records = source_count 
    THEN '✅ Volume consistente com a fonte'
    ELSE '❌ Divergência na contagem de registros'
  END AS count_validation

FROM checks;


total_records,unique_match_ids,source_count,null_dates,null_teams,null_winner,null_match_id,basic_validation,match_id_validation,count_validation
641,641,645,0,0,0,0,✅ Validações básicas passaram,✅ match_id único para cada disputa de pênaltis,❌ Divergência na contagem de registros


In [0]:
%sql
SELECT 
  date(s.date) AS match_date,
  s.home_team,
  s.away_team,
  s.winner,
  s.first_shooter
FROM bronze.shootouts s
WHERE NOT EXISTS (
  SELECT 1
  FROM silver.shootouts ss
  WHERE ss.match_date = date(s.date)
    AND (
      ss.home_team = s.home_team AND ss.away_team = s.away_team
      OR ss.home_team = s.away_team AND ss.away_team = s.home_team
    )
);

match_date,home_team,away_team,winner,first_shooter
1976-06-20,Czechoslovakia,Germany,Czechoslovakia,Czechoslovakia
1980-06-21,Italy,Czechoslovakia,Czechoslovakia,Italy
1991-08-08,Czechoslovakia,Yugoslavia,Czechoslovakia,null
2011-06-29,Saare County,Åland Islands,Åland Islands,null


Durante a construção da tabela silver.shootouts, 4 registros da fonte bronze.shootouts não foram incluídos devido à ausência de correspondência com a tabela silver.results. Isso ocorreu por dois principais motivos:

1. **Seleções extintas**: Como **Czechoslovakia** e **Yugoslavia**, que não constam na silver.results, pois foram substituídas por países atuais e seus nomes antigos não têm correspondência clara nos dados padronizados.

2. **Regiões autônomas ou não reconhecidas como seleções oficiais**: Casos como **Saare County** e **Åland Islands**, que são divisões regionais e não aparecem como seleções na base principal de partidas (silver.results).

Esses registros foram explicitamente excluídos da tabela com base em seus valores de home_team e away_team, a fim de manter a integridade e consistência do modelo dimensional.


In [0]:
%sql
DELETE FROM silver.shootouts
WHERE 
  (match_date = '1976-06-20' AND home_team = 'Czechoslovakia' AND away_team = 'Germany') OR
  (match_date = '1980-06-21' AND home_team = 'Italy' AND away_team = 'Czechoslovakia') OR
  (match_date = '1991-08-08' AND home_team = 'Czechoslovakia' AND away_team = 'Yugoslavia') OR
  (match_date = '2011-06-29' AND home_team = 'Saare County' AND away_team = 'Åland Islands');

num_affected_rows
0


Verificando a exclusão.

In [0]:
%sql
SELECT match_date, home_team, away_team, winner, first_shooter
FROM silver.shootouts
WHERE (match_date = '1976-06-20' AND home_team = 'Czechoslovakia' AND away_team = 'Germany')
   OR (match_date = '1980-06-21' AND home_team = 'Italy' AND away_team = 'Czechoslovakia')
   OR (match_date = '1991-08-08' AND home_team = 'Czechoslovakia' AND away_team = 'Yugoslavia')
   OR (match_date = '2011-06-29' AND home_team = 'Saare County' AND away_team = 'Åland Islands');


match_date,home_team,away_team,winner,first_shooter


In [0]:
%sql
SELECT COUNT(*) AS total_after_delete
FROM silver.shootouts;


total_after_delete
641


O mesmo procedimento de inserção de comentários é utilizado.

In [0]:
%sql
-- Comentário da tabela
ALTER TABLE silver.shootouts 
SET TBLPROPERTIES (
  'comment' = 'Tabela com informações de disputas de pênaltis em partidas internacionais. Utiliza dados limpos e padronizados da camada Bronze com match_id vinculado à tabela silver.results.'
);

-- Comentários das colunas (um comando por coluna)
ALTER TABLE silver.shootouts CHANGE COLUMN match_id match_id STRING 
COMMENT 'Código único da partida, vinculado à tabela silver.results. Domínio: Até 100 caracteres.';

ALTER TABLE silver.shootouts CHANGE COLUMN match_date match_date DATE 
COMMENT 'Data da partida no formato YYYY-MM-DD. Domínio: Datas entre 1872-11-30 e a data atual.';

ALTER TABLE silver.shootouts CHANGE COLUMN home_team home_team STRING 
COMMENT 'Nome padronizado da equipe mandante. Domínio: Países conforme tabela silver.results.';

ALTER TABLE silver.shootouts CHANGE COLUMN away_team away_team STRING 
COMMENT 'Nome padronizado da equipe visitante. Domínio: Países conforme tabela silver.results.';

ALTER TABLE silver.shootouts CHANGE COLUMN winner winner STRING 
COMMENT 'Nome padronizado da equipe que venceu a disputa de pênaltis. Pode ser nulo em registros incompletos.';

ALTER TABLE silver.shootouts CHANGE COLUMN first_shooter first_shooter STRING 
COMMENT 'Equipe que iniciou a disputa de pênaltis. Domínio: Nome da equipe ou "Unknown" caso não informado.';
;

In [0]:
%sql
DESCRIBE TABLE silver.shootouts;

col_name,data_type,comment
match_id,string,"Código único da partida, vinculado à tabela silver.results. Domínio: Até 100 caracteres."
match_date,date,Data da partida no formato YYYY-MM-DD. Domínio: Datas entre 1872-11-30 e a data atual.
home_team,string,Nome padronizado da equipe mandante. Domínio: Países conforme tabela silver.results.
away_team,string,Nome padronizado da equipe visitante. Domínio: Países conforme tabela silver.results.
winner,string,Nome padronizado da equipe que venceu a disputa de pênaltis. Pode ser nulo em registros incompletos.
first_shooter,string,"Equipe que iniciou a disputa de pênaltis. Domínio: Nome da equipe ou ""Unknown"" caso não informado."


### 4.2.4 Tabela Goalscorers
A tabela silver.goalscorers armazena os detalhes dos gols marcados em partidas internacionais. Os dados foram extraídos da camada bronze.goalscorers e passaram por diversas transformações para garantir qualidade e padronização. Os nomes históricos das seleções (colunas team, home_team e away_team) foram padronizados por meio da tabela de correspondência silver.former_names, assegurando consistência com as tabelas da camada Silver. O campo scorer foi preenchido com o valor 'Unknown' nos casos de ausência, garantindo que não existam nulos nessa coluna. O mesmo foi feito para o campo minute, que indica o momento do gol, cujo valor ausente também foi substituído por 'Unknown'. Os campos own_goal e penalty, que indicam respectivamente se o gol foi contra ou de pênalti, foram convertidos explicitamente para o tipo booleano. Para relacionar cada registro de gol com o respectivo jogo, foi realizada uma junção com a tabela silver.results, com base nas colunas home_team, away_team e match_date. Essa etapa foi essencial para incluir o identificador único da partida (match_id), permitindo que os dados sejam analisados no contexto do jogo.

In [0]:
%sql
CREATE OR REPLACE TABLE silver.goalscorers AS
SELECT 
  r.match_id,  
  date(g.date) AS match_date,
  g.team AS scoring_team,
  g.home_team AS home_team,
  g.away_team AS away_team,
  COALESCE(g.scorer, 'Unknown') AS scorer,
  COALESCE(g.minute, 'Unknown') AS minute,
  CAST(g.own_goal AS BOOLEAN) AS is_own_goal,
  CAST(g.penalty AS BOOLEAN) AS is_penalty
FROM (
  SELECT
    g.date,
    COALESCE(fn_team.current, g.team) AS team,
    COALESCE(fn_home.current, g.home_team) AS home_team,
    COALESCE(fn_away.current, g.away_team) AS away_team,
    g.scorer,
    g.minute,
    g.own_goal,
    g.penalty
  FROM bronze.goalscorers g
  LEFT JOIN silver.former_names fn_team 
    ON g.team = fn_team.former
  LEFT JOIN silver.former_names fn_home 
    ON g.home_team = fn_home.former
  LEFT JOIN silver.former_names fn_away 
    ON g.away_team = fn_away.former
) g
LEFT JOIN silver.results r
  ON r.home_team = g.home_team
  AND r.away_team = g.away_team
  AND r.match_date = date(g.date);



num_affected_rows,num_inserted_rows


As transformações são validadas.

In [0]:
%sql
WITH checks AS (
  SELECT
    COUNT(*) AS total_records,

    -- Verificações de campos obrigatórios
    SUM(CASE WHEN match_id IS NULL OR match_id = '' THEN 1 ELSE 0 END) AS null_match_id,
    SUM(CASE WHEN match_date IS NULL THEN 1 ELSE 0 END) AS null_dates,
    SUM(CASE WHEN scoring_team IS NULL THEN 1 ELSE 0 END) AS null_teams,
    SUM(CASE WHEN home_team IS NULL OR away_team IS NULL THEN 1 ELSE 0 END) AS null_home_away_teams,
    SUM(CASE WHEN scorer IS NULL THEN 1 ELSE 0 END) AS null_scorers,
    SUM(CASE WHEN minute IS NULL THEN 1 ELSE 0 END) AS null_minutes,

    -- Verificações de tipos booleanos
    SUM(CASE WHEN is_own_goal IS NULL THEN 1 ELSE 0 END) AS null_own_goal,
    SUM(CASE WHEN is_penalty IS NULL THEN 1 ELSE 0 END) AS null_penalty,

    -- Contagem distinta de match_id + scorer + minute (como proxy de unicidade)
    COUNT(DISTINCT CONCAT(match_id, scorer, minute)) AS unique_goals,

    -- Total da fonte
    (SELECT COUNT(*) FROM bronze.goalscorers) AS source_count

  FROM silver.goalscorers
)

SELECT
  total_records,
  unique_goals,
  source_count,
  null_match_id,
  null_dates,
  null_teams,
  null_home_away_teams,
  null_scorers,
  null_minutes,
  null_own_goal,
  null_penalty,

  -- Validação de integridade
  CASE
    WHEN null_match_id = 0 AND null_dates = 0 AND null_teams = 0 
         AND null_home_away_teams = 0 AND null_scorers = 0 AND null_minutes = 0
         AND null_own_goal = 0 AND null_penalty = 0
    THEN '✅ Validações básicas passaram'
    ELSE '❌ Problemas encontrados nos dados'
  END AS basic_validation,

  -- Verificação de duplicidade de gols
  CASE
    WHEN total_records = unique_goals
    THEN '✅ Nenhuma duplicidade detectada (match_id + scorer + minute)'
    ELSE '❌ Possíveis duplicidades de gols detectadas'
  END AS uniqueness_validation,

  -- Verificação de volume
  CASE
    WHEN total_records = source_count
    THEN '✅ Volume consistente com a fonte'
    ELSE '❌ Divergência na contagem de registros'
  END AS count_validation

FROM checks;


total_records,unique_goals,source_count,null_match_id,null_dates,null_teams,null_home_away_teams,null_scorers,null_minutes,null_own_goal,null_penalty,basic_validation,uniqueness_validation,count_validation
44362,44271,44362,3,0,0,0,0,0,0,0,❌ Problemas encontrados nos dados,❌ Possíveis duplicidades de gols detectadas,✅ Volume consistente com a fonte


Durante o processo de validação da tabela silver.goalscorers, foram identificadas duas inconsistências principais que exigem uma análise mais aprofundada. A primeira refere-se à presença de registros com valores nulos na coluna match_id, o que pode indicar falhas no cruzamento entre os dados de artilheiros e os resultados das partidas.

In [0]:
%sql
-- Verifica se existem partidas na silver.results entre 10 de outbro e 22 de novembro de 1922
SELECT 
  match_date,
  home_team,
  away_team,
  match_id,
  tournament,
  city,
  country
FROM silver.results
WHERE match_date BETWEEN DATE('1922-10-10') AND DATE('1922-11-22')
ORDER BY match_date;


match_date,home_team,away_team,match_id,tournament,city,country
1922-10-12,Paraguay,Uruguay,1922-10-12_Paraguay_Uruguay_1_0,Copa América,Rio de Janeiro,Brazil
1922-10-15,Brazil,Argentina,1922-10-15_Brazil_Argentina_2_0,Copa América,Rio de Janeiro,Brazil
1922-10-18,Argentina,Paraguay,1922-10-18_Argentina_Paraguay_2_0,Copa América,Rio de Janeiro,Brazil
1922-10-21,England,Northern Ireland,1922-10-21_England_Northern Ireland_2_0,British Home Championship,West Bromwich,England
1922-10-22,Argentina,Chile,1922-10-22_Argentina_Chile_1_0,Friendly,Buenos Aires,Argentina
1922-10-22,Brazil,Argentina,1922-10-22_Brazil_Argentina_2_1,Copa Roca,São Paulo,Brazil
1922-10-29,Brazil,Paraguay,1922-10-29_Brazil_Paraguay_3_1,Friendly,São Paulo,Brazil
1922-11-06,Brazil,Paraguay,1922-11-06_Brazil_Paraguay_3_0,Copa América,Rio de Janeiro,Brazil
1922-11-12,Uruguay,Argentina,1922-11-12_Uruguay_Argentina_1_0,Copa Lipton,Montevideo,Uruguay
1922-11-19,Switzerland,Netherlands,1922-11-19_Switzerland_Netherlands_5_0,Friendly,Bern,Switzerland


In [0]:
%sql
CREATE OR REPLACE TABLE silver.goalscorers AS
SELECT 
  r.match_id,  
  r.match_date,  -- Agora usamos a data da tabela results para garantir alinhamento
  g.team AS scoring_team,
  g.home_team,
  g.away_team,
  COALESCE(g.scorer, 'Unknown') AS scorer,
  COALESCE(g.minute, 'Unknown') AS minute,
  CAST(g.own_goal AS BOOLEAN) AS is_own_goal,
  CAST(g.penalty AS BOOLEAN) AS is_penalty
FROM (
  SELECT
    -- Substitui a data incorreta manualmente com base em match_date correto da silver.results
    CASE 
      WHEN g.date = '1922-10-22' 
           AND g.home_team = 'Brazil' AND g.away_team = 'Paraguay'
      THEN '1922-11-06'
      ELSE g.date
    END AS date,
    COALESCE(fn_team.current, g.team) AS team,
    COALESCE(fn_home.current, g.home_team) AS home_team,
    COALESCE(fn_away.current, g.away_team) AS away_team,
    g.scorer,
    g.minute,
    g.own_goal,
    g.penalty
  FROM bronze.goalscorers g
  LEFT JOIN silver.former_names fn_team 
    ON g.team = fn_team.former
  LEFT JOIN silver.former_names fn_home 
    ON g.home_team = fn_home.former
  LEFT JOIN silver.former_names fn_away 
    ON g.away_team = fn_away.former
) g
LEFT JOIN silver.results r
  ON r.home_team = g.home_team
  AND r.away_team = g.away_team
  AND r.match_date = date(g.date);


num_affected_rows,num_inserted_rows


Verifica a correção.

In [0]:
%sql
-- Verifica se os registros corrigidos agora estão com a data 1922-11-06
SELECT *
FROM silver.goalscorers
WHERE match_date = '1922-11-06'
  AND home_team = 'Brazil'
  AND away_team = 'Paraguay'
ORDER BY minute;


match_id,match_date,scoring_team,home_team,away_team,scorer,minute,is_own_goal,is_penalty
1922-11-06_Brazil_Paraguay_3_0,1922-11-06,Brazil,Brazil,Paraguay,Neco,11.0,false,false
1922-11-06_Brazil_Paraguay_3_0,1922-11-06,Brazil,Brazil,Paraguay,Xavier Camargo,48.0,false,false
1922-11-06_Brazil_Paraguay_3_0,1922-11-06,Brazil,Brazil,Paraguay,Xavier Camargo,89.0,false,false


A segunda inconsistência envolve possíveis duplicidades na combinação de campos que caracterizam um gol em uma partida, como o jogador, minuto, time e tipo de gol. Essas duplicidades podem indicar erros na ingestão ou representar situações válidas que precisam ser avaliadas com atenção antes de qualquer tratamento.
A seguir, são identificadas as duplicidades.

In [0]:
%sql
WITH duplicates AS (
  SELECT 
    match_id,
    scoring_team,
    scorer,
    minute,
    is_own_goal,
    is_penalty,
    COUNT(*) AS count
  FROM silver.goalscorers
  GROUP BY 
    match_id,
    scoring_team,
    scorer,
    minute,
    is_own_goal,
    is_penalty
  HAVING count > 1
)

SELECT d.*, g.match_date, g.home_team, g.away_team
FROM duplicates d
JOIN silver.goalscorers g
  ON d.match_id = g.match_id
  AND d.scoring_team = g.scoring_team
  AND d.scorer = g.scorer
  AND d.minute = g.minute
  AND d.is_own_goal = g.is_own_goal
  AND d.is_penalty = g.is_penalty
ORDER BY g.match_date;


match_id,scoring_team,scorer,minute,is_own_goal,is_penalty,count,match_date,home_team,away_team
1963-11-26_Ghana_Ethiopia_2_0,Ghana,Edward Acquah,Unknown,false,false,2,1963-11-26,Ghana,Ethiopia
1963-11-26_Ghana_Ethiopia_2_0,Ghana,Edward Acquah,Unknown,false,false,2,1963-11-26,Ghana,Ethiopia
1963-11-28_Ethiopia_Tunisia_4_2,Ethiopia,Mengistu Worku,Unknown,false,false,2,1963-11-28,Ethiopia,Tunisia
1963-11-28_Nigeria_Sudan_0_4,Sudan,Nasr Eddin Abbas,Unknown,false,false,2,1963-11-28,Nigeria,Sudan
1963-11-28_Ethiopia_Tunisia_4_2,Ethiopia,Mengistu Worku,Unknown,false,false,2,1963-11-28,Ethiopia,Tunisia
1963-11-28_Nigeria_Sudan_0_4,Sudan,Nasr Eddin Abbas,Unknown,false,false,2,1963-11-28,Nigeria,Sudan
1965-03-14_Suriname_Trinidad and Tobago_6_1,Suriname,Edmund Waterval,Unknown,false,false,2,1965-03-14,Suriname,Trinidad and Tobago
1965-03-14_Suriname_Trinidad and Tobago_6_1,Suriname,Edmund Waterval,Unknown,false,false,2,1965-03-14,Suriname,Trinidad and Tobago
1968-01-16_Congo_Ghana_1_3,Ghana,Osei Kofi,Unknown,false,false,2,1968-01-16,Congo,Ghana
1968-01-16_Congo_Ghana_1_3,Ghana,Osei Kofi,Unknown,false,false,2,1968-01-16,Congo,Ghana


Após análise cuidadosa, decidiu-se manter esses registros na tabela, considerando que:

- É comum que um mesmo jogador marque dois ou mais gols em uma mesma partida.
- Em alguns casos, o campo minute não está identificado corretamente (registrado como 'Unknown'), o que dificulta a distinção temporal entre os gols, mas não invalida a ocorrência de múltiplos gols pelo mesmo atleta.
- Gols marcados aos minutos 45 e 90 podem representar tentos nos acréscimos do primeiro e segundo tempo, respectivamente — o que reforça que repetições nesses minutos não significam, necessariamente, duplicidade indevida.

Portanto, os registros foram considerados válidos e foram mantidos como parte íntegra da tabela silver.goalscorers.

Para concluir a criação da tabela, são incluídos os comentários, como nas tabelas anteriores.

In [0]:
%sql
-- Comentário da tabela
ALTER TABLE silver.goalscorers 
SET TBLPROPERTIES (
  'comment' = 'Tabela de artilheiros de partidas internacionais. Registra quem marcou os gols, incluindo penalidades e gols contra. Fonte: bronze.goalscorers, com padronizações e match_id obtido via silver.results.'
);

-- Comentários das colunas (um comando por coluna)
ALTER TABLE silver.goalscorers CHANGE COLUMN match_id match_id STRING 
  COMMENT 'Identificador único da partida, proveniente da tabela silver.results.';

ALTER TABLE silver.goalscorers CHANGE COLUMN match_date match_date DATE 
  COMMENT 'Data da partida no formato YYYY-MM-DD.';

ALTER TABLE silver.goalscorers CHANGE COLUMN scoring_team scoring_team STRING 
  COMMENT 'Nome da equipe que marcou o gol, padronizado conforme a tabela silver.former_names.';

ALTER TABLE silver.goalscorers CHANGE COLUMN home_team home_team STRING 
  COMMENT 'Nome do time mandante na partida, padronizado conforme a tabela silver.former_names.';

ALTER TABLE silver.goalscorers CHANGE COLUMN away_team away_team STRING 
  COMMENT 'Nome do time visitante na partida, padronizado conforme a tabela silver.former_names.';

ALTER TABLE silver.goalscorers CHANGE COLUMN scorer scorer STRING 
  COMMENT 'Nome do jogador que marcou o gol. Domínio: texto livre (ex: "Pelé", "Messi"). Se ausente, recebe "Unknown".';

ALTER TABLE silver.goalscorers CHANGE COLUMN minute minute STRING 
  COMMENT 'Minuto do gol como texto (ex: "45+1", "89"). Se ausente, recebe "Unknown".';

ALTER TABLE silver.goalscorers CHANGE COLUMN is_own_goal is_own_goal BOOLEAN 
  COMMENT 'Indica se o gol foi contra. TRUE para gol contra, FALSE caso contrário.';

ALTER TABLE silver.goalscorers CHANGE COLUMN is_penalty is_penalty BOOLEAN 
  COMMENT 'Indica se o gol foi de pênalti. TRUE para penalidade, FALSE caso contrário.';


In [0]:
%sql
DESCRIBE TABLE silver.goalscorers

col_name,data_type,comment
match_id,string,"Identificador único da partida, proveniente da tabela silver.results."
match_date,date,Data da partida no formato YYYY-MM-DD.
scoring_team,string,"Nome da equipe que marcou o gol, padronizado conforme a tabela silver.former_names."
home_team,string,"Nome do time mandante na partida, padronizado conforme a tabela silver.former_names."
away_team,string,"Nome do time visitante na partida, padronizado conforme a tabela silver.former_names."
scorer,string,"Nome do jogador que marcou o gol. Domínio: texto livre (ex: ""Pelé"", ""Messi""). Se ausente, recebe ""Unknown""."
minute,string,"Minuto do gol como texto (ex: ""45+1"", ""89""). Se ausente, recebe ""Unknown""."
is_own_goal,boolean,"Indica se o gol foi contra. TRUE para gol contra, FALSE caso contrário."
is_penalty,boolean,"Indica se o gol foi de pênalti. TRUE para penalidade, FALSE caso contrário."


### 4.2.5 Tabela FIFA_Ranking
A tabela silver.fifa_ranking foi criada a partir da base bruta bronze.fifa_ranking, contendo os rankings históricos das seleções nacionais publicados pela FIFA entre 1992 e 2024. Durante o processo de transformação, foi realizada a conversão explícita de tipos para os campos de posição no ranking (rank), pontuação total (total_points) e variação no ranking (rank_change) a fim de garantir consistência numérica. Para assegurar a padronização dos nomes das seleções ao longo do tempo, foi aplicada uma junção com a tabela silver.former_names, utilizando uma comparação case-insensitive e ignorando espaços em branco. Também foram incluídas as colunas confederation e rank_change para enriquecer a análise posterior. Por fim, registros com a data de ranking nula foram desconsiderados para garantir integridade temporal.

In [0]:
%sql
DROP TABLE IF EXISTS silver.fifa_ranking;

CREATE TABLE silver.fifa_ranking AS
SELECT 
  date(r.rank_date) AS rank_date,
  COALESCE(fn.current, r.country_full) AS country,
  CAST(r.rank AS INT) AS rank,
  CAST(r.total_points AS DECIMAL(10,2)) AS total_points,
  r.confederation,  -- Adicionado exemplo extra
  CAST(r.rank_change AS INT) AS rank_change  -- Adicionado exemplo extra
FROM bronze.fifa_ranking r
LEFT JOIN silver.former_names fn 
  ON TRIM(UPPER(r.country_full)) = TRIM(UPPER(fn.former))  -- Exemplo: join case-insensitive
WHERE r.rank_date IS NOT NULL;  -- Exemplo: filtrar nulo

num_affected_rows,num_inserted_rows


As transformações são validadas.

In [0]:
%sql
WITH checks AS (
  SELECT
    COUNT(*) AS total_records,

    -- Validação de campos obrigatórios
    SUM(CASE WHEN rank_date IS NULL THEN 1 ELSE 0 END) AS null_dates,
    SUM(CASE WHEN country IS NULL THEN 1 ELSE 0 END) AS null_countries,
    SUM(CASE WHEN rank IS NULL THEN 1 ELSE 0 END) AS null_ranks,
    SUM(CASE WHEN total_points IS NULL THEN 1 ELSE 0 END) AS null_points,

    -- Rank deve ser positivo
    SUM(CASE WHEN rank <= 0 THEN 1 ELSE 0 END) AS invalid_ranks,

    -- Verificação de duplicidade
    COUNT(DISTINCT CONCAT(rank_date, '-', country)) AS unique_entries,

    -- Total original da bronze, após filtro
    (SELECT COUNT(*) FROM bronze.fifa_ranking WHERE rank_date IS NOT NULL) AS source_count

  FROM silver.fifa_ranking
)

SELECT
  total_records,
  unique_entries,
  source_count,
  null_dates,
  null_countries,
  null_ranks,
  null_points,
  invalid_ranks,

  -- Validação geral dos dados
  CASE
    WHEN null_dates = 0 AND null_countries = 0 AND null_ranks = 0 AND null_points = 0 AND invalid_ranks = 0
    THEN '✅ Validações básicas passaram'
    ELSE '❌ Problemas encontrados nos dados'
  END AS basic_validation,

  -- Validação de unicidade (por data e país)
  CASE
    WHEN total_records = unique_entries 
    THEN '✅ Um único ranking por data e país'
    ELSE '❌ Rankings duplicados encontrados'
  END AS uniqueness_validation,

  -- Validação de volume
  CASE
    WHEN total_records = source_count 
    THEN '✅ Volume consistente com a base original'
    ELSE '❌ Divergência na contagem de registros'
  END AS count_validation

FROM checks;


total_records,unique_entries,source_count,null_dates,null_countries,null_ranks,null_points,invalid_ranks,basic_validation,uniqueness_validation,count_validation
67472,67472,67472,0,0,9,0,0,❌ Problemas encontrados nos dados,✅ Um único ranking por data e país,✅ Volume consistente com a base original


Foram encontrados 9 valores null na coluna referente ao ranking. Os mesmos são listados e em seguida é realizada a correção, deletando os mesmos da lista, pois não é um dado possível de ser substituído por algum valor arbitrário.

In [0]:
%sql
SELECT *
FROM silver.fifa_ranking
WHERE rank IS NULL OR rank < 1 OR rank > 216; -- Assumindo que há 211 países no ranking

rank_date,country,rank,total_points,confederation,rank_change
2023-10-26,Eritrea,null,855.56,CAF,0
2023-10-26,Tonga,null,861.81,OFC,0
2023-10-26,Samoa,null,894.26,OFC,0
2023-10-26,American Samoa,null,900.27,OFC,0
2023-11-30,Eritrea,null,855.56,CAF,0
2023-12-21,Eritrea,null,855.56,CAF,0
2024-02-15,Eritrea,null,855.56,CAF,0
2024-04-04,Eritrea,null,855.56,CAF,0
2024-06-20,Eritrea,null,855.56,CAF,0


In [0]:
%sql
DELETE FROM silver.fifa_ranking
WHERE rank IS NULL OR rank < 1 OR rank > 216;

num_affected_rows
9


Verificando a exclusão.

In [0]:
%sql
SELECT *
FROM silver.fifa_ranking
WHERE rank IS NULL OR rank < 1 OR rank > 216; -- Assumindo que há 216 países no ranking

rank_date,country,rank,total_points,confederation,rank_change


Também são atualizados os nomes de alguns países para ficar de acordo com o registrado na tabela results.

In [0]:
%sql
-- Atualiza os nomes dos países na tabela silver.ranking para padronização
UPDATE silver.fifa_ranking
SET country = CASE 
    WHEN country = 'Brunei Darussalam' THEN 'Brunei'
    WHEN country = 'Congo DR' THEN 'DR Congo'
    WHEN country = 'Chinese Taipei' THEN 'Taiwan'
    WHEN country = 'Cabo Verde' THEN 'Cape Verde'
    WHEN country = 'Curacao' THEN 'Curaçao'
    WHEN country = 'Czechia' THEN 'Czech Republic'
    WHEN country = 'The Gambia' THEN 'Gambia'
    WHEN country = 'IR Iran' THEN 'Iran'
    WHEN country = "Côte d'Ivoire" THEN 'Ivory Coast'
    WHEN country = 'Kyrgyz Republic' THEN 'Kyrgyzstan'
    WHEN country = 'Korea DPR' THEN 'North Korea'
    WHEN country = 'St Kitts and Nevis' THEN 'Saint Kitts and Nevis'
    WHEN country = 'St Lucia' THEN 'Saint Lucia'
    WHEN country = 'Korea Republic' THEN 'South Korea'
    WHEN country = 'Sao Tome and Principe' THEN 'São Tomé and Príncipe'
    WHEN country = 'USA' THEN 'United States'
    WHEN country = 'St Vincent and the Grenadines' THEN 'Saint Vincent and the Grenadines'
    WHEN country = 'US Virgin Islands' THEN 'United States Virgin Islands'
    ELSE country
END
WHERE country IN (
    'Brunei Darussalam', 'Congo DR', 'Chinese Taipei', 'Cabo Verde', 
    'Curacao', 'Czechia', 'The Gambia', 'IR Iran', "Côte d'Ivoire", 
    'Korea DPR', 'St Kitts and Nevis', 'St Lucia', 'Korea Republic', 
    'Sao Tome and Principe', 'USA', 'Kyrgyz Republic', 
    'St Vincent and the Grenadines', 'US Virgin Islands'
);

num_affected_rows
5594


In [0]:
%sql
SELECT DISTINCT country FROM silver.fifa_ranking
WHERE country IN ('Brunei', 'DR Congo', 'Taiwan', 'Cape Verde', 'Curaçao', 
                      'Czech Republic', 'Gambia', 'Iran', 'Ivory Coast', 
                      'North Korea', 'Saint Kitts and Nevis', 'Saint Lucia', 
                      'South Korea', 'São Tomé and Príncipe','Kyrgyzstan',
                      'Saint Vincent and the Grenadines', 
                      'United States Virgin Islands')
ORDER BY country;

country
Brunei
Cape Verde
Curaçao
Czech Republic
DR Congo
Gambia
Iran
Ivory Coast
Kyrgyzstan
North Korea


Após todas as correções, são inseridos os comentários. 

In [0]:
%sql
-- Comentário para a tabela
ALTER TABLE silver.fifa_ranking SET TBLPROPERTIES (
  'comment' = 'Tabela de rankings históricos da FIFA, com informações padronizadas sobre data, país, colocação, pontuação total, confederação e variação de posição.'
);

-- Comentários para as colunas com domínio
ALTER TABLE silver.fifa_ranking ALTER COLUMN rank_date COMMENT 
  'Data do ranking FIFA publicado (formato: yyyy-MM-dd). Domínio: 1992-12-31 a 2024-06-20.';

ALTER TABLE silver.fifa_ranking ALTER COLUMN country COMMENT 
  'Nome atual do país, com padronização para nomes históricos. Domínio: aproximadamente 210 países reconhecidos pela FIFA.';

ALTER TABLE silver.fifa_ranking ALTER COLUMN rank COMMENT 
  'Posição do país no ranking FIFA na respectiva data. Domínio: valores inteiros entre 1 (melhor colocado) e cerca de 210.';

ALTER TABLE silver.fifa_ranking ALTER COLUMN total_points COMMENT 
  'Pontuação total do país no ranking FIFA na respectiva data. Domínio: valores decimais positivos, variando conforme fórmula da FIFA.';

ALTER TABLE silver.fifa_ranking ALTER COLUMN confederation COMMENT 
  'Confederação continental à qual o país pertence. Domínio: "UEFA", "CONMEBOL", "CONCACAF", "CAF", "AFC", "OFC".';

ALTER TABLE silver.fifa_ranking ALTER COLUMN rank_change COMMENT 
  'Diferença de posição em relação ao ranking anterior. Domínio: valores inteiros positivos (subida), negativos (queda) ou zero (sem alteração).';


In [0]:
%sql
DESCRIBE TABLE silver.fifa_ranking

col_name,data_type,comment
rank_date,date,Data do ranking FIFA publicado (formato: yyyy-MM-dd). Domínio: 1992-12-31 a 2024-06-20.
country,string,"Nome atual do país, com padronização para nomes históricos. Domínio: aproximadamente 210 países reconhecidos pela FIFA."
rank,int,Posição do país no ranking FIFA na respectiva data. Domínio: valores inteiros entre 1 (melhor colocado) e cerca de 210.
total_points,"decimal(10,2)","Pontuação total do país no ranking FIFA na respectiva data. Domínio: valores decimais positivos, variando conforme fórmula da FIFA."
confederation,string,"Confederação continental à qual o país pertence. Domínio: ""UEFA"", ""CONMEBOL"", ""CONCACAF"", ""CAF"", ""AFC"", ""OFC""."
rank_change,int,"Diferença de posição em relação ao ranking anterior. Domínio: valores inteiros positivos (subida), negativos (queda) ou zero (sem alteração)."


A camada silver do projeto ficou composta por cinco tabelas resultantes do tratamento, padronização e enriquecimento dos dados brutos da camada bronze. Quatro dessas tabelas compõem o núcleo da estrutura, enquanto uma foi utilizada exclusivamente como apoio no processo de transformação.

As tabelas foram organizadas seguindo um esquema `flat`, ou seja, não há definição explícita de relacionamentos entre elas nesta camada. Cada tabela é autônoma e autossuficiente, refletindo diferentes aspectos dos dados históricos do futebol internacional. Esse modelo foi adotado para garantir simplicidade, flexibilidade e performance na leitura dos dados, deixando a integração entre entidades para ser realizada de forma contextualizada na camada gold.


Essa abordagem flat garante uma base limpa, segmentada e de fácil manutenção para a construção da camada gold, onde os relacionamentos entre entidades serão formalizados conforme os objetivos analíticos do projeto.


Com a camada silver concluída, segue-se agora para a camada gold, onde as tabelas serão modeladas com agregações e métricas que permitirão responder às perguntas de negócio e extrair insights sobre a dominância histórica das seleções nacionais no futebol.

## 4.3 Camada Gold

A camada Gold representa o estágio final do pipeline de dados, onde as informações limpas e padronizadas da camada Silver são transformadas em modelos analíticos prontos para consumo. Essa etapa é fundamental para a construção de análises históricas e comparativas, permitindo extrair insights sobre o desempenho das seleções nacionais de futebol.

Para estruturar essa camada, foi adotado o `modelo estrela (star schema)`, um padrão amplamente utilizado em soluções de Business Intelligence e Data Warehousing. Esse modelo organiza os dados em uma tabela fato central conectada a múltiplas tabelas dimensão, promovendo desempenho, flexibilidade e simplicidade na execução de consultas analíticas.

O modelo da camada Gold foi projetado para permitir a realização de análises detalhadas por partida, seleção, jogador e disputa de pênaltis, apoiando a construção de visualizações e dashboards com diferentes recortes.

A estrutura da camada Gold é composta por:

- **1 tabela fato**: `gold.match_performance`
- **3 tabelas dimensão**: `gold.team_stats`, `gold.penalty_shootouts` e `gold.top_scores`

Essa organização permite a centralização dos dados das partidas na tabela fato e o enriquecimento da análise por meio de diferentes perspectivas oferecidas pelas tabelas dimensão.

Nos próximos blocos, serão apresentadas as características de cada tabela, os critérios utilizados em sua construção e os relacionamentos definidos entre elas.


In [0]:
%sql DROP DATABASE IF EXISTS gold CASCADE

In [0]:
%sql CREATE DATABASE gold

### 4.3.1 Tabela Fato: `gold.match_performance`

A tabela `match_performance` representa o núcleo da camada Gold e armazena informações detalhadas sobre cada partida registrada na base de dados. Ela concentra os principais indicadores de desempenho de cada confronto, servindo como base para relacionar com as tabelas dimensão.

**Principais atributos:**
-`match_id`: Identificador único da partida, utilizado para integrar diferentes fontes de dados relacionadas ao mesmo jogo.
- `match_date`: Data em que a partida foi realizada.
- `home_team`: Nome da seleção que atuou como mandante.
- `away_team`: Nome da seleção adversária, que atuou como visitante.
- `home_score`: Número de gols marcados pela seleção mandante.
- `away_score`: Número de gols marcados pela seleção visitante.
- `home_team_rank`: Posição da seleção mandante no ranking da FIFA imediatamente anterior à data da partida.
- `away_team_rank`: Posição da seleção visitante no ranking da FIFA imediatamente anterior à data da partida.
- `rank_diff`: Diferença de posições no ranking entre a seleção mandante e a visitante (home_team_rank - away_team_rank), útil para análises comparativas de desempenho.
- `home_team_confederation`: Confederação à qual pertence a seleção mandante (ex: UEFA, CONMEBOL, etc.).
- `away_team_confederation`: Confederação à qual pertence a seleção visitante.
- `tournament`: Tipo de competição em que a partida foi disputada, categorizada como "Friendly" para amistosos ou "Competitive" para torneios oficiais.
- `is_neutral`: Indicador booleano que identifica se a partida foi realizada em campo neutro (valor true) ou com vantagem de mando (valor false).

A tabela `gold.match_performance` foi construída a partir da junção dos dados de partidas com os rankings FIFA mais recentes disponíveis antes da realização de cada jogo. O processo envolveu a identificação da última data de ranking válida para cada seleção (mandante e visitante), com base na data do jogo, garantindo que as análises levem em consideração o contexto competitivo de cada confronto. A partir disso, foram incluídas informações como posição no ranking, confederação, resultado da partida e se o jogo ocorreu em campo neutro.







In [0]:
%sql
CREATE OR REPLACE TABLE gold.match_performance AS
WITH latest_ranking AS (
    SELECT 
        country, 
        rank_date, 
        rank, 
        confederation,
        ROW_NUMBER() OVER (
            PARTITION BY country, rank_date 
            ORDER BY rank_date DESC
        ) AS rn
    FROM silver.fifa_ranking
),

home_ranking AS (
    SELECT 
        r.match_id, 
        MAX(fr.rank_date) AS latest_rank_date
    FROM silver.results r
    JOIN silver.fifa_ranking fr
        ON r.home_team = fr.country
        AND fr.rank_date <= r.match_date
    GROUP BY r.match_id
),

away_ranking AS (
    SELECT 
        r.match_id, 
        MAX(fr.rank_date) AS latest_rank_date
    FROM silver.results r
    JOIN silver.fifa_ranking fr
        ON r.away_team = fr.country
        AND fr.rank_date <= r.match_date
    GROUP BY r.match_id
)

SELECT 
    r.match_id,
    r.match_date,
    r.home_team,
    r.away_team,
    r.home_score,
    r.away_score,
    hr.rank AS home_team_rank,
    ar.rank AS away_team_rank,
    COALESCE(hr.rank - ar.rank, 0) AS rank_diff,
    hr.confederation AS home_team_confederation,
    ar.confederation AS away_team_confederation,
    r.tournament,
    r.is_neutral
FROM silver.results r
LEFT JOIN home_ranking h
    ON r.match_id = h.match_id
LEFT JOIN away_ranking a
    ON r.match_id = a.match_id
LEFT JOIN latest_ranking hr
    ON hr.country = r.home_team 
    AND hr.rank_date = h.latest_rank_date
LEFT JOIN latest_ranking ar
    ON ar.country = r.away_team 
    AND ar.rank_date = a.latest_rank_date;



num_affected_rows,num_inserted_rows


A tabela é visualizada após a criação.

In [0]:
%sql
SELECT *
FROM gold.match_performance
ORDER BY match_date DESC
LIMIT 20

match_id,match_date,home_team,away_team,home_score,away_score,home_team_rank,away_team_rank,rank_diff,home_team_confederation,away_team_confederation,tournament,is_neutral
2025-03-25_Martinique_Suriname_0_1,2025-03-25,Martinique,Suriname,0,1,null,141,0,null,CONCACAF,Gold Cup qualification,false
2025-03-25_Algeria_Mozambique_5_1,2025-03-25,Algeria,Mozambique,5,1,44,103,-59,CAF,CAF,FIFA World Cup qualification,false
2025-03-25_Sweden_Northern Ireland_5_1,2025-03-25,Sweden,Northern Ireland,5,1,28,73,-45,UEFA,UEFA,Friendly,false
2025-03-25_Bolivia_Uruguay_0_0,2025-03-25,Bolivia,Uruguay,0,0,84,14,70,CONMEBOL,CONMEBOL,FIFA World Cup qualification,false
2025-03-25_Liechtenstein_Kazakhstan_0_2,2025-03-25,Liechtenstein,Kazakhstan,0,2,199,109,90,UEFA,UEFA,FIFA World Cup qualification,false
2025-03-25_Mauritania_DR Congo_0_2,2025-03-25,Mauritania,DR Congo,0,2,112,61,51,CAF,CAF,FIFA World Cup qualification,false
2025-03-25_Guatemala_Guyana_2_0,2025-03-25,Guatemala,Guyana,2,0,106,153,-47,CONCACAF,CONCACAF,Gold Cup qualification,false
2025-03-25_Israel_Norway_2_4,2025-03-25,Israel,Norway,2,4,79,46,33,UEFA,UEFA,FIFA World Cup qualification,true
2025-03-25_Switzerland_Luxembourg_3_1,2025-03-25,Switzerland,Luxembourg,3,1,19,85,-66,UEFA,UEFA,Friendly,false
2025-03-25_Kuwait_Oman_0_1,2025-03-25,Kuwait,Oman,0,1,137,76,61,AFC,AFC,FIFA World Cup qualification,false


Como na camada silver, são inseridos comentários para a tabela e suas respectivas colunas.

In [0]:
%sql
-- Comentário da tabela
ALTER TABLE gold.match_performance 
SET TBLPROPERTIES (
  'comment' = 'Tabela fato com informações detalhadas sobre o desempenho das seleções em partidas internacionais. Baseada em dados tratados da camada Silver com enriquecimento a partir do ranking FIFA.'
);

-- Comentários das colunas (um comando por coluna)
ALTER TABLE gold.match_performance CHANGE COLUMN match_id match_id STRING 
COMMENT 'Identificador único da partida. Domínio: Até 100 caracteres, exemplo: BRA-ARG-1994-07-17.';

ALTER TABLE gold.match_performance CHANGE COLUMN match_date match_date DATE 
COMMENT 'Data em que a partida foi realizada. Domínio: Datas entre 1872-11-30 e a data atual.';

ALTER TABLE gold.match_performance CHANGE COLUMN home_team home_team STRING 
COMMENT 'Nome padronizado da seleção mandante. Domínio: Países conforme tabela silver.results.';

ALTER TABLE gold.match_performance CHANGE COLUMN away_team away_team STRING 
COMMENT 'Nome padronizado da seleção visitante. Domínio: Países conforme tabela silver.results.';

ALTER TABLE gold.match_performance CHANGE COLUMN home_score home_score INT 
COMMENT 'Gols marcados pela seleção mandante. Domínio: Números inteiros >= 0.';

ALTER TABLE gold.match_performance CHANGE COLUMN away_score away_score INT 
COMMENT 'Gols marcados pela seleção visitante. Domínio: Números inteiros >= 0.';

ALTER TABLE gold.match_performance CHANGE COLUMN home_team_rank home_team_rank INT 
COMMENT 'Ranking FIFA da seleção mandante antes da partida. Domínio: 1 a 211.';

ALTER TABLE gold.match_performance CHANGE COLUMN away_team_rank away_team_rank INT 
COMMENT 'Ranking FIFA da seleção visitante antes da partida. Domínio: 1 a 211.';

ALTER TABLE gold.match_performance CHANGE COLUMN rank_diff rank_diff INT 
COMMENT 'Diferença entre os rankings FIFA das seleções (mandante - visitante). Domínio: Pode ser positivo, negativo ou zero.';

ALTER TABLE gold.match_performance CHANGE COLUMN home_team_confederation home_team_confederation STRING 
COMMENT 'Confederação da seleção mandante. Domínio: UEFA, CONMEBOL, CONCACAF, AFC, CAF, OFC.';

ALTER TABLE gold.match_performance CHANGE COLUMN away_team_confederation away_team_confederation STRING 
COMMENT 'Confederação da seleção visitante. Domínio: UEFA, CONMEBOL, CONCACAF, AFC, CAF, OFC.';

ALTER TABLE gold.match_performance CHANGE COLUMN tournament tournament STRING 
COMMENT 'Nome do torneio em que a partida foi disputada. Domínio: Exemplos incluem "FIFA World Cup", "Friendly", "Copa América", etc.';

ALTER TABLE gold.match_performance CHANGE COLUMN is_neutral is_neutral BOOLEAN 
COMMENT 'Indica se a partida foi em campo neutro. Domínio: true (sim) ou false (não).';


In [0]:
%sql
DESCRIBE TABLE gold.match_performance

col_name,data_type,comment
match_id,string,"Identificador único da partida. Domínio: Até 100 caracteres, exemplo: BRA-ARG-1994-07-17."
match_date,date,Data em que a partida foi realizada. Domínio: Datas entre 1872-11-30 e a data atual.
home_team,string,Nome padronizado da seleção mandante. Domínio: Países conforme tabela silver.results.
away_team,string,Nome padronizado da seleção visitante. Domínio: Países conforme tabela silver.results.
home_score,int,Gols marcados pela seleção mandante. Domínio: Números inteiros >= 0.
away_score,int,Gols marcados pela seleção visitante. Domínio: Números inteiros >= 0.
home_team_rank,int,Ranking FIFA da seleção mandante antes da partida. Domínio: 1 a 211.
away_team_rank,int,Ranking FIFA da seleção visitante antes da partida. Domínio: 1 a 211.
rank_diff,int,"Diferença entre os rankings FIFA das seleções (mandante - visitante). Domínio: Pode ser positivo, negativo ou zero."
home_team_confederation,string,"Confederação da seleção mandante. Domínio: UEFA, CONMEBOL, CONCACAF, AFC, CAF, OFC."



### 4.3.2 Tabela Dimensão: `gold.team_stats`

A tabela `team_stats` consolida estatísticas históricas das seleções, permitindo uma visão agregada do desempenho de cada equipe nacional ao longo do tempo. Ela inclui métricas clássicas de performance e eficiência.

**Principais atributos:**
- `team_id`: Identificador único da seleção nacional.  
- `team_name`: Nome oficial da seleção.  
- `confederation`: Confederação à qual a seleção está afiliada (ex: UEFA, CONMEBOL, AFC, etc.).  
- `total_matches`: Número total de partidas disputadas pela seleção ao longo do período analisado.  
- `wins`: Total de partidas vencidas.  
- `draws`: Total de partidas empatadas.  
- `losses`: Total de partidas perdidas.  
- `goals_for`: Número total de gols marcados pela seleção.  
- `goals_against`: Número total de gols sofridos.  
- `goal_difference`: Saldo de gols da seleção, calculado como `goals_for - goals_against`.  
- `clean_sheets`: Quantidade de jogos em que a seleção não sofreu gols.  
- `points`: Pontuação total acumulada, considerando 3 pontos por vitória e 1 por empate.  
- `win_percentage`: Percentual de aproveitamento da seleção, calculado com base na proporção de vitórias sobre o total de partidas disputadas.

In [0]:
%sql
CREATE OR REPLACE TABLE gold.team_stats AS
WITH latest_confederation AS (
    SELECT 
        country, 
        confederation,
        ROW_NUMBER() OVER (PARTITION BY country ORDER BY rank_date DESC) AS rn
    FROM silver.fifa_ranking
)
SELECT
    t.team AS team_id,
    t.team AS team_name,
    COALESCE(lc.confederation, 'Unknown') AS confederation,
    COUNT(*) AS total_matches,
    SUM(CASE WHEN t.goals_for > t.goals_against THEN 1 ELSE 0 END) AS wins,
    SUM(CASE WHEN t.goals_for = t.goals_against THEN 1 ELSE 0 END) AS draws,
    SUM(CASE WHEN t.goals_for < t.goals_against THEN 1 ELSE 0 END) AS losses,
    SUM(t.goals_for) AS goals_for,
    SUM(t.goals_against) AS goals_against,
    SUM(t.goals_for) - SUM(t.goals_against) AS goal_difference,
    SUM(CASE WHEN t.goals_against = 0 THEN 1 ELSE 0 END) AS clean_sheets,
    (SUM(CASE WHEN t.goals_for > t.goals_against THEN 3 ELSE 0 END) + 
     SUM(CASE WHEN t.goals_for = t.goals_against THEN 1 ELSE 0 END)) AS points,
    ROUND(
        SUM(CASE WHEN t.goals_for > t.goals_against THEN 1 ELSE 0 END) * 100.0 / COUNT(*),
        2
    ) AS win_percentage
FROM (
    SELECT home_team AS team, home_score AS goals_for, away_score AS goals_against FROM silver.results
    UNION ALL
    SELECT away_team AS team, away_score AS goals_for, home_score AS goals_against FROM silver.results
) AS t
LEFT JOIN latest_confederation lc
    ON t.team = lc.country
    AND lc.rn = 1  -- Pegamos apenas a confederação mais recente
GROUP BY t.team, lc.confederation;


num_affected_rows,num_inserted_rows


Visualização da tabela criada.

In [0]:
%sql
SELECT* 
FROM gold.team_stats
ORDER BY total_matches DESC
LIMIT 20

team_id,team_name,confederation,total_matches,wins,draws,losses,goals_for,goals_against,goal_difference,clean_sheets,points,win_percentage
Sweden,Sweden,UEFA,1089,537,230,322,2152,1398,754,357,1841,49.31
England,England,UEFA,1078,616,257,205,2355,1036,1319,446,2105,57.14
Argentina,Argentina,CONMEBOL,1055,581,256,218,1995,1070,925,410,1999,55.07
Brazil,Brazil,CONMEBOL,1047,665,214,168,2279,946,1333,467,2209,63.51
Germany,Germany,UEFA,1019,590,213,216,2291,1186,1105,350,1983,57.90
South Korea,South Korea,AFC,995,530,251,214,1771,902,869,459,1841,53.27
Hungary,Hungary,UEFA,994,466,219,309,1992,1468,524,278,1617,46.88
Mexico,Mexico,CONCACAF,982,503,224,255,1736,1036,700,402,1733,51.22
Uruguay,Uruguay,CONMEBOL,956,421,235,300,1514,1166,348,323,1498,44.04
France,France,UEFA,923,468,194,261,1684,1195,489,331,1598,50.70


Inserção de comentários descritivos.

In [0]:
%sql
-- Comentário da tabela
ALTER TABLE gold.team_stats 
SET TBLPROPERTIES (
  'comment' = 'Tabela dimensão que consolida estatísticas históricas agregadas de cada seleção nacional. Inclui informações como número de partidas, vitórias, empates, derrotas, gols marcados e sofridos, saldo de gols, clean sheets, pontos e aproveitamento. A confederação é associada com base no ranking FIFA mais recente.'
);

-- Comentários das colunas (um comando por coluna)

ALTER TABLE gold.team_stats CHANGE COLUMN team_id team_id STRING 
COMMENT 'Identificador único da seleção nacional. Domínio: Nome padronizado dos países conforme tabela silver.results.';

ALTER TABLE gold.team_stats CHANGE COLUMN team_name team_name STRING 
COMMENT 'Nome padronizado da seleção nacional. Domínio: Mesmo valor de team_id.';

ALTER TABLE gold.team_stats CHANGE COLUMN confederation confederation STRING 
COMMENT 'Confederação mais recente à qual a seleção pertence (ex: UEFA, CONMEBOL, CAF, AFC, CONCACAF, OFC). Domínio: Lista de confederações da FIFA ou "Unknown".';

ALTER TABLE gold.team_stats CHANGE COLUMN total_matches total_matches BIGINT
COMMENT 'Número total de partidas disputadas pela seleção. Domínio: Inteiros positivos.';

ALTER TABLE gold.team_stats CHANGE COLUMN wins wins BIGINT 
COMMENT 'Total de partidas vencidas pela seleção. Domínio: Inteiros positivos, sempre menor ou igual a total_matches.';

ALTER TABLE gold.team_stats CHANGE COLUMN draws draws BIGINT 
COMMENT 'Total de partidas empatadas pela seleção. Domínio: Inteiros positivos, sempre menor ou igual a total_matches.';

ALTER TABLE gold.team_stats CHANGE COLUMN losses losses BIGINT 
COMMENT 'Total de partidas perdidas pela seleção. Domínio: Inteiros positivos, sempre menor ou igual a total_matches.';

ALTER TABLE gold.team_stats CHANGE COLUMN goals_for goals_for BIGINT 
COMMENT 'Total de gols marcados pela seleção. Domínio: Inteiros positivos.';

ALTER TABLE gold.team_stats CHANGE COLUMN goals_against goals_against BIGINT 
COMMENT 'Total de gols sofridos pela seleção. Domínio: Inteiros positivos.';

ALTER TABLE gold.team_stats CHANGE COLUMN goal_difference goal_difference BIGINT 
COMMENT 'Saldo de gols da seleção (gols marcados - gols sofridos). Domínio: Números inteiros (positivos, negativos ou zero).';

ALTER TABLE gold.team_stats CHANGE COLUMN clean_sheets clean_sheets BIGINT 
COMMENT 'Número de partidas em que a seleção não sofreu gols. Domínio: Inteiros positivos, menor ou igual a total_matches.';

ALTER TABLE gold.team_stats CHANGE COLUMN points points BIGINT 
COMMENT 'Pontuação total baseada na regra de 3 pontos por vitória e 1 ponto por empate. Domínio: Inteiros positivos.';

ALTER TABLE gold.team_stats CHANGE COLUMN win_percentage win_percentage DECIMAL(27,2) 
COMMENT 'Percentual de vitórias da seleção em relação ao total de partidas. Domínio: Valores entre 0.00 e 100.00.';


In [0]:
%sql
DESCRIBE TABLE gold.team_stats

col_name,data_type,comment
team_id,string,Identificador único da seleção nacional. Domínio: Nome padronizado dos países conforme tabela silver.results.
team_name,string,Nome padronizado da seleção nacional. Domínio: Mesmo valor de team_id.
confederation,string,"Confederação mais recente à qual a seleção pertence (ex: UEFA, CONMEBOL, CAF, AFC, CONCACAF, OFC). Domínio: Lista de confederações da FIFA ou ""Unknown""."
total_matches,bigint,Número total de partidas disputadas pela seleção. Domínio: Inteiros positivos.
wins,bigint,"Total de partidas vencidas pela seleção. Domínio: Inteiros positivos, sempre menor ou igual a total_matches."
draws,bigint,"Total de partidas empatadas pela seleção. Domínio: Inteiros positivos, sempre menor ou igual a total_matches."
losses,bigint,"Total de partidas perdidas pela seleção. Domínio: Inteiros positivos, sempre menor ou igual a total_matches."
goals_for,bigint,Total de gols marcados pela seleção. Domínio: Inteiros positivos.
goals_against,bigint,Total de gols sofridos pela seleção. Domínio: Inteiros positivos.
goal_difference,bigint,"Saldo de gols da seleção (gols marcados - gols sofridos). Domínio: Números inteiros (positivos, negativos ou zero)."




### 4.3.3 Tabela Dimensão: `gold.top_scores`

A tabela `top_scores` destaca os principais artilheiros da base de dados, vinculando o desempenho individual dos jogadores às respectivas seleções.

**Principais atributos:**
- `player_name`: Nome do jogador.
- `team_name`: Nome da seleção pela qual o jogador marcou os gols.
- `total_goals`: Total de gols marcados pelo jogador (excluindo gols contra).
- `matches_scored`: Número de partidas diferentes em que o jogador marcou pelo menos um gol.
- `max_goals_in_a_game`: Maior número de gols marcados pelo jogador em uma única partida.
- `hat_tricks`: Quantidade de vezes que o jogador marcou 3 ou mais gols em um mesmo jogo.
- `penalty_goals`: Total de gols marcados de pênalti.
- `own_goals`: Total de gols contra registrados pelo jogador (a favor da equipe adversária).

Com essa dimensão, é possível analisar o impacto dos principais jogadores no desempenho ofensivo das seleções.

In [0]:
%sql
-- Criação da tabela gold.top_scorers com métricas adicionais

CREATE OR REPLACE TABLE gold.top_scorers AS
WITH gols_por_partida AS (
    SELECT
        scorer,
        scoring_team,
        match_id,
        COUNT(*) AS total_gols
    FROM silver.goalscorers
    WHERE is_own_goal = false
    GROUP BY scorer, scoring_team, match_id
),
hat_tricks_cte AS (
    SELECT
        scorer,
        scoring_team,
        COUNT(*) AS hat_tricks
    FROM gols_por_partida
    WHERE total_gols >= 3
    GROUP BY scorer, scoring_team
),
tipo_gols_cte AS (
    SELECT
        scorer,
        scoring_team,
        SUM(CASE WHEN is_penalty THEN 1 ELSE 0 END) AS penalty_goals,
        SUM(CASE WHEN is_own_goal THEN 1 ELSE 0 END) AS own_goals
    FROM silver.goalscorers
    GROUP BY scorer, scoring_team
)
SELECT
    gpp.scorer AS player_name,
    gpp.scoring_team AS team_name,
    SUM(gpp.total_gols) AS total_goals,
    COUNT(DISTINCT gpp.match_id) AS matches_scored,
    MAX(gpp.total_gols) AS max_goals_in_a_game,
    COALESCE(ht.hat_tricks, 0) AS hat_tricks,
    COALESCE(tg.penalty_goals, 0) AS penalty_goals,
    COALESCE(tg.own_goals, 0) AS own_goals
FROM gols_por_partida gpp
LEFT JOIN hat_tricks_cte ht
    ON gpp.scorer = ht.scorer AND gpp.scoring_team = ht.scoring_team
LEFT JOIN tipo_gols_cte tg
    ON gpp.scorer = tg.scorer AND gpp.scoring_team = tg.scoring_team
GROUP BY
    gpp.scorer,
    gpp.scoring_team,
    ht.hat_tricks,
    tg.penalty_goals,
    tg.own_goals;


num_affected_rows,num_inserted_rows


Visualização da tabela criada.

In [0]:
%sql
SELECT *
FROM gold.top_scorers
ORDER BY total_goals DESC
LIMIT 20

player_name,team_name,total_goals,matches_scored,max_goals_in_a_game,hat_tricks,penalty_goals,own_goals
Cristiano Ronaldo,Portugal,108,67,4,10,19,0
Robert Lewandowski,Poland,63,40,4,6,13,0
Romelu Lukaku,Belgium,60,37,4,3,6,0
Harry Kane,England,58,42,4,5,20,0
Lionel Messi,Argentina,55,44,3,3,14,0
Edin Džeko,Bosnia and Herzegovina,50,38,3,2,2,0
Ali Daei,Iran,49,34,4,3,8,0
Miroslav Klose,Germany,48,34,3,2,0,0
Luis Suárez,Uruguay,47,37,4,1,9,0
Aleksandar Mitrović,Serbia,47,32,3,2,2,0


Inserção de comentários descritivos.

In [0]:
%sql
-- Comentário geral da tabela
ALTER TABLE gold.top_scorers 
SET TBLPROPERTIES (
  'comment' = 'Tabela com os principais artilheiros da história das seleções, baseada na contagem consolidada de gols por jogador a partir da tabela silver.goalscorers.'
);

-- Comentários das colunas

ALTER TABLE gold.top_scorers CHANGE COLUMN player_name player_name STRING 
COMMENT 'Nome do jogador que marcou gols em partidas internacionais. Domínio: até 100 caracteres.';

ALTER TABLE gold.top_scorers CHANGE COLUMN team_name team_name STRING 
COMMENT 'Nome da seleção pela qual o jogador marcou gols. Domínio: nomes padronizados de países presentes na tabela silver.goalscorers.';

ALTER TABLE gold.top_scorers CHANGE COLUMN total_goals total_goals BIGINT 
COMMENT 'Quantidade total de gols marcados pelo jogador (excluindo gols contra). Domínio: inteiros positivos.';

ALTER TABLE gold.top_scorers CHANGE COLUMN matches_scored matches_scored BIGINT 
COMMENT 'Número de partidas diferentes em que o jogador marcou pelo menos um gol. Domínio: inteiros positivos.';

ALTER TABLE gold.top_scorers CHANGE COLUMN max_goals_in_a_game max_goals_in_a_game BIGINT 
COMMENT 'Maior número de gols marcados pelo jogador em uma única partida. Domínio: inteiros positivos.';

ALTER TABLE gold.top_scorers CHANGE COLUMN hat_tricks hat_tricks BIGINT 
COMMENT 'Número de jogos em que o jogador marcou 3 ou mais gols (hat-tricks). Domínio: inteiros positivos.';

ALTER TABLE gold.top_scorers CHANGE COLUMN penalty_goals penalty_goals BIGINT 
COMMENT 'Total de gols marcados de pênalti pelo jogador. Domínio: inteiros positivos.';

ALTER TABLE gold.top_scorers CHANGE COLUMN own_goals own_goals BIGINT 
COMMENT 'Total de gols contra registrados pelo jogador, a favor da equipe adversária. Domínio: inteiros positivos.';


In [0]:
%sql
DESCRIBE TABLE gold.top_scorers

col_name,data_type,comment
player_name,string,Nome do jogador que marcou gols em partidas internacionais. Domínio: até 100 caracteres.
team_name,string,Nome da seleção pela qual o jogador marcou gols. Domínio: nomes padronizados de países presentes na tabela silver.goalscorers.
total_goals,bigint,Quantidade total de gols marcados pelo jogador (excluindo gols contra). Domínio: inteiros positivos.
matches_scored,bigint,Número de partidas diferentes em que o jogador marcou pelo menos um gol. Domínio: inteiros positivos.
max_goals_in_a_game,bigint,Maior número de gols marcados pelo jogador em uma única partida. Domínio: inteiros positivos.
hat_tricks,bigint,Número de jogos em que o jogador marcou 3 ou mais gols (hat-tricks). Domínio: inteiros positivos.
penalty_goals,bigint,Total de gols marcados de pênalti pelo jogador. Domínio: inteiros positivos.
own_goals,bigint,"Total de gols contra registrados pelo jogador, a favor da equipe adversária. Domínio: inteiros positivos."


### 4.3.4 Tabela Dimensão: `gold.penalty_shootouts`

A tabela `penalty_shootouts` traz informações detalhadas sobre disputas de pênaltis registradas na base de dados, focando nos aspectos decisivos desse tipo de confronto.

**Principais atributos:**
- `match_id`: Identificador da partida
- `match_date`: Data do jogo
- `home_team`: Seleção mandante
- `away_team`: Seleção visitante
- `penalty_winner`: Seleção vencedora da disputa
- `penalty_loser`: Seleção perdedora da disputa
- `first_penalty_shooter`: Seleção que iniciou a disputa de pênaltis

Essa tabela possibilita análises específicas sobre o desempenho das seleções em decisões por pênaltis.

In [0]:
%sql
CREATE OR REPLACE TABLE gold.penalty_shootouts AS
SELECT
    s.match_id,
    s.match_date,
    s.home_team,
    s.away_team,
    s.winner AS penalty_winner,
    CASE 
        WHEN s.winner = s.home_team THEN s.away_team
        ELSE s.home_team
    END AS penalty_loser,
    s.first_shooter AS first_penalty_shooter
FROM silver.shootouts s;


num_affected_rows,num_inserted_rows


Visualização da tabela.

In [0]:
%sql
SELECT *
FROM gold.penalty_shootouts
LIMIT 20

match_id,match_date,home_team,away_team,penalty_winner,penalty_loser,first_penalty_shooter
1967-08-22_India_Taiwan_1_1,1967-08-22,India,Taiwan,Taiwan,India,Unknown
1971-11-14_South Korea_Vietnam Republic_1_1,1971-11-14,South Korea,Vietnam Republic,South Korea,Vietnam Republic,Unknown
1972-05-07_South Korea_Iraq_0_0,1972-05-07,South Korea,Iraq,Iraq,South Korea,Unknown
1972-05-17_Thailand_South Korea_1_1,1972-05-17,Thailand,South Korea,South Korea,Thailand,Unknown
1972-05-19_Thailand_Cambodia_2_2,1972-05-19,Thailand,Cambodia,Thailand,Cambodia,Unknown
1973-04-21_Senegal_Ghana_1_0,1973-04-21,Senegal,Ghana,Ghana,Senegal,Unknown
1973-06-14_Guinea_Mali_1_1,1973-06-14,Guinea,Mali,Guinea,Mali,Unknown
1973-06-14_Mauritius_Tanzania_1_1,1973-06-14,Mauritius,Tanzania,Mauritius,Tanzania,Unknown
1973-07-26_Cambodia_Singapore_1_1,1973-07-26,Cambodia,Singapore,Singapore,Cambodia,Unknown
1973-07-26_Malaysia_Kuwait_0_0,1973-07-26,Malaysia,Kuwait,Malaysia,Kuwait,Unknown


Inserção de comentários descritivos.

In [0]:
%sql
-- Comentário da tabela
ALTER TABLE gold.penalty_shootouts 
SET TBLPROPERTIES (
  'comment' = 'Tabela com informações de disputas de pênaltis em partidas internacionais, derivadas da camada Silver. Inclui detalhes sobre as seleções envolvidas, vencedores e ordem dos cobradores.'
);

-- Comentários das colunas (um comando por coluna)
ALTER TABLE gold.penalty_shootouts CHANGE COLUMN match_id match_id STRING 
COMMENT 'Identificador único da partida, vinculado à tabela silver.shootouts. Domínio: Até 100 caracteres.';

ALTER TABLE gold.penalty_shootouts CHANGE COLUMN match_date match_date DATE 
COMMENT 'Data da partida no formato YYYY-MM-DD. Domínio: Datas entre 1872-11-30 e a data atual.';

ALTER TABLE gold.penalty_shootouts CHANGE COLUMN home_team home_team STRING 
COMMENT 'Nome padronizado da seleção mandante. Domínio: Países conforme tabela silver.shootouts.';

ALTER TABLE gold.penalty_shootouts CHANGE COLUMN away_team away_team STRING 
COMMENT 'Nome padronizado da seleção visitante. Domínio: Países conforme tabela silver.shootouts.';

ALTER TABLE gold.penalty_shootouts CHANGE COLUMN penalty_winner penalty_winner STRING 
COMMENT 'Seleção que venceu a disputa de pênaltis. Domínio: Nome de um dos times da partida ou nulo em registros incompletos.';

ALTER TABLE gold.penalty_shootouts CHANGE COLUMN penalty_loser penalty_loser STRING 
COMMENT 'Seleção que perdeu a disputa de pênaltis. Domínio: Nome de um dos times da partida ou nulo em registros incompletos.';

ALTER TABLE gold.penalty_shootouts CHANGE COLUMN first_penalty_shooter first_penalty_shooter STRING 
COMMENT 'Seleção que iniciou a disputa de pênaltis. Domínio: Nome de uma das seleções da partida ou "Unknown" se não disponível.';


In [0]:
%sql
DESCRIBE TABLE gold.penalty_shootouts


col_name,data_type,comment
match_id,string,"Identificador único da partida, vinculado à tabela silver.shootouts. Domínio: Até 100 caracteres."
match_date,date,Data da partida no formato YYYY-MM-DD. Domínio: Datas entre 1872-11-30 e a data atual.
home_team,string,Nome padronizado da seleção mandante. Domínio: Países conforme tabela silver.shootouts.
away_team,string,Nome padronizado da seleção visitante. Domínio: Países conforme tabela silver.shootouts.
penalty_winner,string,Seleção que venceu a disputa de pênaltis. Domínio: Nome de um dos times da partida ou nulo em registros incompletos.
penalty_loser,string,Seleção que perdeu a disputa de pênaltis. Domínio: Nome de um dos times da partida ou nulo em registros incompletos.
first_penalty_shooter,string,"Seleção que iniciou a disputa de pênaltis. Domínio: Nome de uma das seleções da partida ou ""Unknown"" se não disponível."



### 4.4 Relacionamentos do Modelo

A estrutura do modelo estrela garante que todas as tabelas se conectam de forma eficiente por meio da tabela fato `match_performance`.

**Relacionamentos principais:**
- `match_performance` com `team_stats`: via `home_team` e `away_team`
- `match_performance` com `penalty_shootouts`: via `match_id`
- `match_performance` com `top_scores`: via `match_id`

A escolha do modelo estrela se baseia nos seguintes benefícios:
- Facilita a execução de análises complexas e agregações por seleção, jogador e tipo de partida
- Garante alta performance em consultas mesmo com grandes volumes de dados
- Oferece flexibilidade para expansão futura do modelo com novas dimensões
- Simplifica o entendimento e a manutenção da estrutura analítica


A seguir, o esquema relacional pode ser visualizado. Esse esquema foi desenhado utilizando a ferramenta Quick Data Base Diagrams (link: https://www.quickdatabasediagrams.com).

In [0]:
from IPython.display import Image, display

# URL da imagem em versão raw do GitHub
image_url = "https://raw.githubusercontent.com/MarcelLopes7/Engenharia-de-Dados/main/QuickDBD-Esquema%20Estrela%20Camada%20Gold.png"

# Exibe a imagem no notebook
display(Image(url=image_url))

A construção da camada gold consolidou e refinou os dados históricos do futebol em um esquema otimizado para análise. As tabelas foram projetadas para capturar estatísticas detalhadas das seleções, desempenho em partidas, artilheiros e disputas de pênaltis, garantindo consistência e eficiência na consulta dos dados. Além disso, a simplificação do modelo permitiu integrar estatísticas diretamente nas tabelas de desempenho, reduzindo redundâncias sem comprometer a granularidade da análise. Com essa estrutura, as perguntas analíticas poderão ser respondidas de forma mais direta, apoiando insights sobre a dominância e evolução das seleções ao longo do tempo.

A resposta das perguntas com suas respectivas análises, além da conclusão do trabalho estão no notebook MVP - Engenharia de Dados - Análise e Solução do Problema que está contido no mesmo repositório desse notebook.